# Predicting All-NBA Team and Player Salaries - Data Cleaning
---

In this notebook, we will be cleaning and feature engineering our webscraped NBA data to create a final cleaned file for exploratory data analysis.

We will use a combination of Python and SQL to preprocess the data for accuracy and consistency, including handling missing values, addressing data inconsistencies, and converting data into appropriate formats. We will engineer columns such as Voter Share for the All-NBA Team award winners and total NBA payroll per season, as well as narrow down the dataframes to only variables of interest, before conducting a series of merges to create cleaned and final data for modeling. Our exploratory data analysis will give us an overall sense of the makeup of our data and explore various relationships between variables.

Further detailed notebooks on the various segments of this project can be found at the following: 
- [01_Data_Acquisition](./01_Data_Acquisition.ipynb)
- [03_Preliminary_EDA](./03_Preliminary_EDA.ipynb)
- [04_Data_Modeling_I](./04_Data_Modeling_I.ipynb)
- [05_Data_Modeling_II](./05_Data_Modeling_II.ipynb)

For more information on the background, a summary of methods, and findings, please see the associated [README](../README.md) for this analysis. 

### Contents
- [I. Baseline: Combine Player Statistics](#I.-Baseline:-Combine-Player-Statistics)
    - [Initial Cleaning](#Initial-Cleaning)
    - [Dummify Position](#Dummify-Position)
    - [Addressing Players With Mid-Season Transfers](#Addressing-Players-With-Mid-Season-Transfers)
    - [Merge All-NBA Winner Indicators](#Merge-All-NBA-Winner-Indicators)
    - [Assess Missing Values, Null Values, and Outliers](#Assess-Missing-Values,-Null-Values,-and-Outliers)
- [II. Setting Minimum Criteria Using Domain Knowledge](#II.-Setting-Minimum-Criteria-Using-Domain-Knowledge)
    - [Games Played, Minutes, Points, and Player Efficiency](#Games-Played,-Minutes,-Points,-and-Player-Efficiency)
    - [Re-Assess Missing Values](#Re-Assess-Missing-Values)
- [III. Finalizing Player Information](#III.-Finalizing-Player-Information)
    - [Incorporating All-Star Appearance](#Incorporating-All-Star-Appearance)
    - [Integrating Player Salaries](#Integrating-Player-Salaries)
- [IV. Processing Team Information](#IV.-Processing-Team-Information)
    - [Conference and Win Percentages](#Conference-and-Win-Percentages)
    - [Championship Winners](#Championship-Winners)
    - [Salary Cap](#Salary-Cap)
- [V. Merge Player and Team Data](#Merge-Player-and-Team-Data)    

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

import warnings
warnings.filterwarnings('ignore') 

# this setting widens how many characters pandas will display in a column:
#pd.options.display.max_colwidth = 400
pd.options.display.max_rows = 400
pd.options.display.max_columns = 400

In [2]:
adv = pd.read_csv('../data/advanced_data.csv')
pg = pd.read_csv('../data/per_game_data.csv')
tot = pd.read_csv('../data/totals_data.csv')

allteam = pd.read_csv('../data/all_nba_teams.csv')
allstar = pd.read_csv('../data/all_star_appearances.csv')
rank = pd.read_csv('../data/team_rank.csv')

sal = pd.read_csv('../data/salaries.csv')
salcap = pd.read_csv('../data/salarycap.csv')
payroll = pd.read_csv('../data/team_payroll.csv')

## I. Baseline: Combine Player Statistics

In [3]:
print(adv.Pos.value_counts())
print(adv.dtypes)

PF          3903
SG          3827
PG          3745
C           3723
SF          3442
Pos          712
SF-SG         35
SG-SF         31
PG-SG         31
C-PF          28
SG-PG         28
PF-SF         27
PF-C          26
SF-PF         24
SG-PF          4
PG-SF          1
SF-C           1
SG-PG-SF       1
Name: Pos, dtype: int64
Rk              object
Player          object
Pos             object
Age             object
Tm              object
G               object
MP              object
PER             object
TS%             object
3PAr            object
FTr             object
ORB%            object
DRB%            object
TRB%            object
AST%            object
STL%            object
BLK%            object
TOV%            object
USG%            object
Unnamed: 19    float64
OWS             object
DWS             object
WS              object
WS/48           object
Unnamed: 24    float64
OBPM            object
DBPM            object
BPM             object
VORP            object
Yea

In [4]:
print(tot.Pos.value_counts())
print(tot.dtypes)

PF          3903
SG          3827
PG          3745
C           3723
SF          3442
Pos          712
SF-SG         35
SG-SF         31
PG-SG         31
C-PF          28
SG-PG         28
PF-SF         27
PF-C          26
SF-PF         24
SG-PF          4
PG-SF          1
SF-C           1
SG-PG-SF       1
Name: Pos, dtype: int64
Rk        object
Player    object
Pos       object
Age       object
Tm        object
G         object
GS        object
MP        object
FG        object
FGA       object
FG%       object
3P        object
3PA       object
3P%       object
2P        object
2PA       object
2P%       object
eFG%      object
FT        object
FTA       object
FT%       object
ORB       object
DRB       object
TRB       object
AST       object
STL       object
BLK       object
TOV       object
PF        object
PTS       object
Year       int64
Stat      object
dtype: object


In [5]:
print(pg.Pos.value_counts())
print(pg.dtypes)

PF          3903
SG          3827
PG          3745
C           3723
SF          3442
Pos          712
SF-SG         35
SG-SF         31
PG-SG         31
C-PF          28
SG-PG         28
PF-SF         27
PF-C          26
SF-PF         24
SG-PF          4
PG-SF          1
SF-C           1
SG-PG-SF       1
Name: Pos, dtype: int64
Rk        object
Player    object
Pos       object
Age       object
Tm        object
G         object
GS        object
MP        object
FG        object
FGA       object
FG%       object
3P        object
3PA       object
3P%       object
2P        object
2PA       object
2P%       object
eFG%      object
FT        object
FTA       object
FT%       object
ORB       object
DRB       object
TRB       object
AST       object
STL       object
BLK       object
TOV       object
PF        object
PTS       object
Year       int64
Stat      object
dtype: object


### Initial Cleaning
##### Some initial data cleaning tasks have become apparent:
- Remove any Unnamed rows
- Remove rows which are repeat headers
- Convert Objects to Floats
- Dummify ```Pos``` (Position)
- Prefix variables with statistic type

### Remove Erroneous Columns/Rows, Update Data Types, Assign Prefixes

In [6]:
def init_clean(df, pre):
    
    # Remove Unnamed columns
    df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

    # Remove repeat header rows
    df = df.loc[df.Rk != 'Rk']

    # Convert to float
    keep_obj = ['Pos', 'Player', 'Tm', 'Stat']
    data_types = {col: 'float64' for col in df.columns if col not in keep_obj}
    df = df.astype(data_types).drop(columns=['Stat'])
    
    # Overwrite float to make these variables integers
    keep_obj2 = ['Rk', 'Year', 'Age', 'G', 'MP'] 
    data_types2 = {col: 'int64' for col in df.columns if col in keep_obj2}
    df = df.astype(data_types2)

    # Dummify Position - will address later
    # df_dum = pd.get_dummies(df['Pos'], drop_first=True)
    # df = pd.concat([df, df_dum], axis=1)

    # Prefix variables
    no_pre = ['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'Year']
    prefix = pre
    rename_col = {col: f'{prefix}_{col}' for col in df.columns if col not in no_pre}
    df = df.rename(columns=rename_col)
    
    return df

In [7]:
adv = init_clean(adv, 'adv')
adv

,Rk,Player,Pos,Age,Tm,G,adv_MP,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,Year
0,1,Alaa Abdelnaby,PF,22,POR,43,290,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2,1990
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,1505,12.2,0.448,0.099,0.097,1.9,6.0,3.8,19.2,1.5,0.1,9.5,27.2,-0.7,-0.3,-1.0,-0.031,-2.0,-3.0,-5.0,-1.1,1990
2,3,Mark Acres,C,28,ORL,68,1313,9.2,0.551,0.014,0.472,11.3,18.7,14.9,2.5,0.9,1.1,14.0,9.3,1.4,1.1,2.5,0.090,-2.8,-0.2,-3.0,-0.3,1990
3,4,Michael Adams,PG,28,DEN,66,2346,22.3,0.530,0.397,0.372,2.1,8.8,5.2,39.4,2.6,0.1,12.7,28.5,5.8,0.4,6.3,0.128,6.0,-0.7,5.3,4.3,1990
4,5,Mark Aguirre,SF,31,DET,78,2006,16.7,0.526,0.086,0.349,7.6,13.7,10.7,11.6,1.2,0.6,10.9,25.7,2.8,2.7,5.5,0.132,1.2,0.2,1.4,1.7,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19584,535,Thaddeus Young,PF,34,TOR,54,795,14.1,0.573,0.172,0.131,9.4,14.6,11.8,12.9,3.4,0.6,16.7,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,2022
19585,536,Trae Young,PG,24,ATL,73,2541,22.0,0.573,0.331,0.460,2.4,7.0,4.7,42.5,1.5,0.3,15.2,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,2022
19586,537,Omer Yurtseven,C,24,MIA,9,83,16.7,0.675,0.259,0.222,10.9,21.9,16.2,3.9,1.2,2.5,11.9,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,2022
19587,538,Cody Zeller,C,30,MIA,15,217,16.4,0.659,0.034,0.593,13.0,21.8,17.3,7.2,0.7,1.9,15.8,18.1,0.4,0.3,0.7,0.147,-2.0,-0.7,-2.8,0.0,2022


In [8]:
tot = init_clean(tot, 'tot')
tot

,Rk,Player,Pos,Age,Tm,G,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,Year
0,1,Alaa Abdelnaby,PF,22,POR,43,0.0,290,55.0,116.0,0.474,0.0,0.0,NaN,55.0,116.0,0.474,0.474,25.0,44.0,0.568,27.0,62.0,89.0,12.0,4.0,12.0,22.0,39.0,135.0,1990
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19.0,1505,417.0,1009.0,0.413,24.0,100.0,0.240,393.0,909.0,0.432,0.425,84.0,98.0,0.857,34.0,87.0,121.0,206.0,55.0,4.0,110.0,149.0,942.0,1990
2,3,Mark Acres,C,28,ORL,68,0.0,1313,109.0,214.0,0.509,1.0,3.0,0.333,108.0,211.0,0.512,0.512,66.0,101.0,0.653,140.0,219.0,359.0,25.0,25.0,25.0,42.0,218.0,285.0,1990
3,4,Michael Adams,PG,28,DEN,66,66.0,2346,560.0,1421.0,0.394,167.0,564.0,0.296,393.0,857.0,0.459,0.453,465.0,529.0,0.879,58.0,198.0,256.0,693.0,147.0,6.0,240.0,162.0,1752.0,1990
4,5,Mark Aguirre,SF,31,DET,78,13.0,2006,420.0,909.0,0.462,24.0,78.0,0.308,396.0,831.0,0.477,0.475,240.0,317.0,0.757,134.0,240.0,374.0,139.0,47.0,20.0,128.0,209.0,1104.0,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19584,535,Thaddeus Young,PF,34,TOR,54,9.0,795,108.0,198.0,0.545,6.0,34.0,0.176,102.0,164.0,0.622,0.561,18.0,26.0,0.692,71.0,95.0,166.0,75.0,54.0,5.0,42.0,88.0,240.0,2022
19585,536,Trae Young,PG,24,ATL,73,73.0,2541,597.0,1390.0,0.429,154.0,460.0,0.335,443.0,930.0,0.476,0.485,566.0,639.0,0.886,56.0,161.0,217.0,741.0,80.0,9.0,300.0,104.0,1914.0,2022
19586,537,Omer Yurtseven,C,24,MIA,9,0.0,83,16.0,27.0,0.593,3.0,7.0,0.429,13.0,20.0,0.650,0.648,5.0,6.0,0.833,8.0,15.0,23.0,2.0,2.0,2.0,4.0,16.0,40.0,2022
19587,538,Cody Zeller,C,30,MIA,15,2.0,217,37.0,59.0,0.627,0.0,2.0,0.000,37.0,57.0,0.649,0.627,24.0,35.0,0.686,25.0,39.0,64.0,10.0,3.0,4.0,14.0,33.0,98.0,2022


In [9]:
pg = init_clean(pg, 'pg')
pg

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year
0,1,Alaa Abdelnaby,PF,22,POR,43,0.0,6,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1990
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19.0,22,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1990
2,3,Mark Acres,C,28,ORL,68,0.0,19,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1990
3,4,Michael Adams,PG,28,DEN,66,66.0,35,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1990
4,5,Mark Aguirre,SF,31,DET,78,13.0,25,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19584,535,Thaddeus Young,PF,34,TOR,54,9.0,14,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2022
19585,536,Trae Young,PG,24,ATL,73,73.0,34,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2022
19586,537,Omer Yurtseven,C,24,MIA,9,0.0,9,1.8,3.0,0.593,0.3,0.8,0.429,1.4,2.2,0.650,0.648,0.6,0.7,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,2022
19587,538,Cody Zeller,C,30,MIA,15,2.0,14,2.5,3.9,0.627,0.0,0.1,0.000,2.5,3.8,0.649,0.627,1.6,2.3,0.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,2022


In [10]:
adv.dtypes

Rk             int64
Player        object
Pos           object
Age            int64
Tm            object
G              int64
adv_MP         int64
adv_PER      float64
adv_TS%      float64
adv_3PAr     float64
adv_FTr      float64
adv_ORB%     float64
adv_DRB%     float64
adv_TRB%     float64
adv_AST%     float64
adv_STL%     float64
adv_BLK%     float64
adv_TOV%     float64
adv_USG%     float64
adv_OWS      float64
adv_DWS      float64
adv_WS       float64
adv_WS/48    float64
adv_OBPM     float64
adv_DBPM     float64
adv_BPM      float64
adv_VORP     float64
Year           int64
dtype: object

In [11]:
tot.dtypes

Rk            int64
Player       object
Pos          object
Age           int64
Tm           object
G             int64
tot_GS      float64
tot_MP        int64
tot_FG      float64
tot_FGA     float64
tot_FG%     float64
tot_3P      float64
tot_3PA     float64
tot_3P%     float64
tot_2P      float64
tot_2PA     float64
tot_2P%     float64
tot_eFG%    float64
tot_FT      float64
tot_FTA     float64
tot_FT%     float64
tot_ORB     float64
tot_DRB     float64
tot_TRB     float64
tot_AST     float64
tot_STL     float64
tot_BLK     float64
tot_TOV     float64
tot_PF      float64
tot_PTS     float64
Year          int64
dtype: object

In [12]:
pg.dtypes

Rk           int64
Player      object
Pos         object
Age          int64
Tm          object
G            int64
pg_GS      float64
pg_MP        int64
pg_FG      float64
pg_FGA     float64
pg_FG%     float64
pg_3P      float64
pg_3PA     float64
pg_3P%     float64
pg_2P      float64
pg_2PA     float64
pg_2P%     float64
pg_eFG%    float64
pg_FT      float64
pg_FTA     float64
pg_FT%     float64
pg_ORB     float64
pg_DRB     float64
pg_TRB     float64
pg_AST     float64
pg_STL     float64
pg_BLK     float64
pg_TOV     float64
pg_PF      float64
pg_PTS     float64
Year         int64
dtype: object

##### <span style = 'color:mediumvioletred'> _We will merge the three datasets by Rk, Player, Pos, Age, Tm, G, as these should be the same across all three datasets._ </span>

In [13]:
stats1 = pg.merge(tot, how='left', on=['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'Year'])
stats1

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS
0,1,Alaa Abdelnaby,PF,22,POR,43,0.0,6,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1990,0.0,290,55.0,116.0,0.474,0.0,0.0,NaN,55.0,116.0,0.474,0.474,25.0,44.0,0.568,27.0,62.0,89.0,12.0,4.0,12.0,22.0,39.0,135.0
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19.0,22,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1990,19.0,1505,417.0,1009.0,0.413,24.0,100.0,0.240,393.0,909.0,0.432,0.425,84.0,98.0,0.857,34.0,87.0,121.0,206.0,55.0,4.0,110.0,149.0,942.0
2,3,Mark Acres,C,28,ORL,68,0.0,19,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1990,0.0,1313,109.0,214.0,0.509,1.0,3.0,0.333,108.0,211.0,0.512,0.512,66.0,101.0,0.653,140.0,219.0,359.0,25.0,25.0,25.0,42.0,218.0,285.0
3,4,Michael Adams,PG,28,DEN,66,66.0,35,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1990,66.0,2346,560.0,1421.0,0.394,167.0,564.0,0.296,393.0,857.0,0.459,0.453,465.0,529.0,0.879,58.0,198.0,256.0,693.0,147.0,6.0,240.0,162.0,1752.0
4,5,Mark Aguirre,SF,31,DET,78,13.0,25,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1990,13.0,2006,420.0,909.0,0.462,24.0,78.0,0.308,396.0,831.0,0.477,0.475,240.0,317.0,0.757,134.0,240.0,374.0,139.0,47.0,20.0,128.0,209.0,1104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18872,535,Thaddeus Young,PF,34,TOR,54,9.0,14,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2022,9.0,795,108.0,198.0,0.545,6.0,34.0,0.176,102.0,164.0,0.622,0.561,18.0,26.0,0.692,71.0,95.0,166.0,75.0,54.0,5.0,42.0,88.0,240.0
18873,536,Trae Young,PG,24,ATL,73,73.0,34,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2022,73.0,2541,597.0,1390.0,0.429,154.0,460.0,0.335,443.0,930.0,0.476,0.485,566.0,639.0,0.886,56.0,161.0,217.0,741.0,80.0,9.0,300.0,104.0,1914.0
18874,537,Omer Yurtseven,C,24,MIA,9,0.0,9,1.8,3.0,0.593,0.3,0.8,0.429,1.4,2.2,0.650,0.648,0.6,0.7,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,2022,0.0,83,16.0,27.0,0.593,3.0,7.0,0.429,13.0,20.0,0.650,0.648,5.0,6.0,0.833,8.0,15.0,23.0,2.0,2.0,2.0,4.0,16.0,40.0
18875,538,Cody Zeller,C,30,MIA,15,2.0,14,2.5,3.9,0.627,0.0,0.1,0.000,2.5,3.8,0.649,0.627,1.6,2.3,0.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,2022,2.0,217,37.0,59.0,0.627,0.0,2.0,0.000,37.0,57.0,0.649,0.627,24.0,35.0,0.686,25.0,39.0,64.0,10.0,3.0,4.0,14.0,33.0,98.0


In [14]:
stats2 = stats1.merge(adv, how='left', on=['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'Year']).drop(columns=['adv_MP']) # Excluding adv_MP because this is the same as tot_MP
stats2

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP
0,1,Alaa Abdelnaby,PF,22,POR,43,0.0,6,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1990,0.0,290,55.0,116.0,0.474,0.0,0.0,NaN,55.0,116.0,0.474,0.474,25.0,44.0,0.568,27.0,62.0,89.0,12.0,4.0,12.0,22.0,39.0,135.0,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19.0,22,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1990,19.0,1505,417.0,1009.0,0.413,24.0,100.0,0.240,393.0,909.0,0.432,0.425,84.0,98.0,0.857,34.0,87.0,121.0,206.0,55.0,4.0,110.0,149.0,942.0,12.2,0.448,0.099,0.097,1.9,6.0,3.8,19.2,1.5,0.1,9.5,27.2,-0.7,-0.3,-1.0,-0.031,-2.0,-3.0,-5.0,-1.1
2,3,Mark Acres,C,28,ORL,68,0.0,19,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1990,0.0,1313,109.0,214.0,0.509,1.0,3.0,0.333,108.0,211.0,0.512,0.512,66.0,101.0,0.653,140.0,219.0,359.0,25.0,25.0,25.0,42.0,218.0,285.0,9.2,0.551,0.014,0.472,11.3,18.7,14.9,2.5,0.9,1.1,14.0,9.3,1.4,1.1,2.5,0.090,-2.8,-0.2,-3.0,-0.3
3,4,Michael Adams,PG,28,DEN,66,66.0,35,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1990,66.0,2346,560.0,1421.0,0.394,167.0,564.0,0.296,393.0,857.0,0.459,0.453,465.0,529.0,0.879,58.0,198.0,256.0,693.0,147.0,6.0,240.0,162.0,1752.0,22.3,0.530,0.397,0.372,2.1,8.8,5.2,39.4,2.6,0.1,12.7,28.5,5.8,0.4,6.3,0.128,6.0,-0.7,5.3,4.3
4,5,Mark Aguirre,SF,31,DET,78,13.0,25,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1990,13.0,2006,420.0,909.0,0.462,24.0,78.0,0.308,396.0,831.0,0.477,0.475,240.0,317.0,0.757,134.0,240.0,374.0,139.0,47.0,20.0,128.0,209.0,1104.0,16.7,0.526,0.086,0.349,7.6,13.7,10.7,11.6,1.2,0.6,10.9,25.7,2.8,2.7,5.5,0.132,1.2,0.2,1.4,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18872,535,Thaddeus Young,PF,34,TOR,54,9.0,14,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2022,9.0,795,108.0,198.0,0.545,6.0,34.0,0.176,102.0,164.0,0.622,0.561,18.0,26.0,0.692,71.0,95.0,166.0,75.0,54.0,5.0,42.0,88.0,240.0,14.1,0.573,0.172,0.131,9.4,14.6,11.8,12.9,3.4,0.6,16.7,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4
18873,536,Trae Young,PG,24,ATL,73,73.0,34,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2022,73.0,2541,597.0,1390.0,0.429,154.0,460.0,0.335,443.0,930.0,0.476,0.485,566.0,639.0,0.886,56.0,161.0,217.0,741.0,80.0,9.0,300.0,104.0,1914.0,22.0,0.573,0.331,0.460,2.4,7.0,4.7,42.5,1.5,0.3,15.2,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4
18874,537,Omer Yurtseven,C,24,MIA,9,0.0,9,1.8,3.0,0.593,0.3,0.8,0.429,1.4,2.2,0.650,0.648,0.6,0.7,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,2022,0.0,83,16.0,27.0,0.593,3.0,7.0,0.429,13.0,20.0,0.650,0.648,5.0,6.0,0.833,8.0,15.0,23.0,2.0,2.0,2.0,4.0,16.0,40.0,16.7,0.675,0.259,0.222,10.9,21.9,16.2,3.9,1.2,2.5,11.9,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0
18875,538,Cody Zeller,C,30,MIA,15,2.0,14,2.5,3.9,0.627,0.0,0.1,0.000,2.5,3.8,0.649,0.627,1.6,2.3,0.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,2022,2.0,217,37.0,59.0,0.627,0.0,2.0

In [15]:
print(f'PerGame Stats Shape: {pg.shape}')
print(f'Total Stats Shape: {tot.shape}')
print(f'Advanced Stats Shape: {adv.shape}')
print(f'First Mrg Shape: {stats1.shape}')
print(f'Final Mrg Shape: {stats2.shape}')
      
# We expect the number of rows to be the same in all, and columns in Stats2, ultimately, to increase by ([columns-in-tot] - 7) +  ([columns-in-adv] - 7 - 1) 

PerGame Stats Shape: (18877, 31)
Total Stats Shape: (18877, 31)
Advanced Stats Shape: (18877, 28)
First Mrg Shape: (18877, 55)
Final Mrg Shape: (18877, 75)


### Dummify Position

In [16]:
adv.Pos.value_counts()

PF          3903
SG          3827
PG          3745
C           3723
SF          3442
SF-SG         35
SG-SF         31
PG-SG         31
C-PF          28
SG-PG         28
PF-SF         27
PF-C          26
SF-PF         24
SG-PF          4
PG-SF          1
SF-C           1
SG-PG-SF       1
Name: Pos, dtype: int64

In [17]:
tot.Pos.value_counts()

PF          3903
SG          3827
PG          3745
C           3723
SF          3442
SF-SG         35
SG-SF         31
PG-SG         31
C-PF          28
SG-PG         28
PF-SF         27
PF-C          26
SF-PF         24
SG-PF          4
PG-SF          1
SF-C           1
SG-PG-SF       1
Name: Pos, dtype: int64

In [18]:
pg.Pos.value_counts()

PF          3903
SG          3827
PG          3745
C           3723
SF          3442
SF-SG         35
SG-SF         31
PG-SG         31
C-PF          28
SG-PG         28
PF-SF         27
PF-C          26
SF-PF         24
SG-PF          4
PG-SF          1
SF-C           1
SG-PG-SF       1
Name: Pos, dtype: int64

In [19]:
adv.Pos.value_counts(normalize=True)

PF          0.206760
SG          0.202733
PG          0.198390
C           0.197224
SF          0.182338
SF-SG       0.001854
SG-SF       0.001642
PG-SG       0.001642
C-PF        0.001483
SG-PG       0.001483
PF-SF       0.001430
PF-C        0.001377
SF-PF       0.001271
SG-PF       0.000212
PG-SF       0.000053
SF-C        0.000053
SG-PG-SF    0.000053
Name: Pos, dtype: float64

##### <span style = 'color:mediumvioletred'> _Only 1.26% of players have more than one position listed. We will take the primary position but will create another variable called "GT1_Pos" to indicate whether a player had more than one position listed. We may use this later as it indicates versatility._ </span>

In [20]:
def cnt_pos(col):
    return col.count('-')

stats2['GT1_Pos'] = stats2.Pos.apply(cnt_pos)
stats2

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,GT1_Pos
0,1,Alaa Abdelnaby,PF,22,POR,43,0.0,6,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1990,0.0,290,55.0,116.0,0.474,0.0,0.0,NaN,55.0,116.0,0.474,0.474,25.0,44.0,0.568,27.0,62.0,89.0,12.0,4.0,12.0,22.0,39.0,135.0,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2,0
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19.0,22,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1990,19.0,1505,417.0,1009.0,0.413,24.0,100.0,0.240,393.0,909.0,0.432,0.425,84.0,98.0,0.857,34.0,87.0,121.0,206.0,55.0,4.0,110.0,149.0,942.0,12.2,0.448,0.099,0.097,1.9,6.0,3.8,19.2,1.5,0.1,9.5,27.2,-0.7,-0.3,-1.0,-0.031,-2.0,-3.0,-5.0,-1.1,0
2,3,Mark Acres,C,28,ORL,68,0.0,19,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1990,0.0,1313,109.0,214.0,0.509,1.0,3.0,0.333,108.0,211.0,0.512,0.512,66.0,101.0,0.653,140.0,219.0,359.0,25.0,25.0,25.0,42.0,218.0,285.0,9.2,0.551,0.014,0.472,11.3,18.7,14.9,2.5,0.9,1.1,14.0,9.3,1.4,1.1,2.5,0.090,-2.8,-0.2,-3.0,-0.3,0
3,4,Michael Adams,PG,28,DEN,66,66.0,35,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1990,66.0,2346,560.0,1421.0,0.394,167.0,564.0,0.296,393.0,857.0,0.459,0.453,465.0,529.0,0.879,58.0,198.0,256.0,693.0,147.0,6.0,240.0,162.0,1752.0,22.3,0.530,0.397,0.372,2.1,8.8,5.2,39.4,2.6,0.1,12.7,28.5,5.8,0.4,6.3,0.128,6.0,-0.7,5.3,4.3,0
4,5,Mark Aguirre,SF,31,DET,78,13.0,25,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1990,13.0,2006,420.0,909.0,0.462,24.0,78.0,0.308,396.0,831.0,0.477,0.475,240.0,317.0,0.757,134.0,240.0,374.0,139.0,47.0,20.0,128.0,209.0,1104.0,16.7,0.526,0.086,0.349,7.6,13.7,10.7,11.6,1.2,0.6,10.9,25.7,2.8,2.7,5.5,0.132,1.2,0.2,1.4,1.7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18872,535,Thaddeus Young,PF,34,TOR,54,9.0,14,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2022,9.0,795,108.0,198.0,0.545,6.0,34.0,0.176,102.0,164.0,0.622,0.561,18.0,26.0,0.692,71.0,95.0,166.0,75.0,54.0,5.0,42.0,88.0,240.0,14.1,0.573,0.172,0.131,9.4,14.6,11.8,12.9,3.4,0.6,16.7,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,0
18873,536,Trae Young,PG,24,ATL,73,73.0,34,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2022,73.0,2541,597.0,1390.0,0.429,154.0,460.0,0.335,443.0,930.0,0.476,0.485,566.0,639.0,0.886,56.0,161.0,217.0,741.0,80.0,9.0,300.0,104.0,1914.0,22.0,0.573,0.331,0.460,2.4,7.0,4.7,42.5,1.5,0.3,15.2,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,0
18874,537,Omer Yurtseven,C,24,MIA,9,0.0,9,1.8,3.0,0.593,0.3,0.8,0.429,1.4,2.2,0.650,0.648,0.6,0.7,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,2022,0.0,83,16.0,27.0,0.593,3.0,7.0,0.429,13.0,20.0,0.650,0.648,5.0,6.0,0.833,8.0,15.0,23.0,2.0,2.0,2.0,4.0,16.0,40.0,16.7,0.675,0.259,0.222,10.9,21.9,16.2,3.9,1.2,2.5,11.9,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,0
18875,538,Cody Zeller,C,30,MIA,15,2.0,14,2.5,3.9,0.627,0.0,0.1,0.000,2.5,3.8,0.649,0.627,1.6,2.3,0.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,2022,2.0

In [21]:
stats2.GT1_Pos.value_counts()

0    18640
1      236
2        1
Name: GT1_Pos, dtype: int64

In [22]:
stats2.GT1_Pos.value_counts(normalize=True)

0    0.987445
1    0.012502
2    0.000053
Name: GT1_Pos, dtype: float64

In [23]:
# Condense Pos variable to only the 5 standard NBA positions: PF, SG, PG, C, SF
stats2['Pos_5'] = stats2['Pos'].apply(lambda x: x.split('-')[0])

In [24]:
stats2.Pos_5.value_counts()

PF    3956
SG    3891
PG    3777
C     3751
SF    3502
Name: Pos_5, dtype: int64

In [25]:
stats2.Pos_5.value_counts(normalize=True)

PF    0.209567
SG    0.206124
PG    0.200085
C     0.198707
SF    0.185517
Name: Pos_5, dtype: float64

In [26]:
# Will dummify vs. OHE, because these positions will not change (i.e., have more added or some taken away) and our data should always contain a mix of each
pos_dummy = pd.get_dummies(stats2['Pos_5'], drop_first=True)
pos_dummy

,PF,PG,SF,SG
0,1,0,0,0
1,0,1,0,0
2,0,0,0,0
3,0,1,0,0
4,0,0,1,0
...,...,...,...,...
18872,1,0,0,0
18873,0,1,0,0
18874,0,0,0,0
18875,0,0,0,0


In [27]:
stats2 = pd.concat([stats2, pos_dummy], axis=1)
stats2

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,GT1_Pos,Pos_5,PF,PG,SF,SG
0,1,Alaa Abdelnaby,PF,22,POR,43,0.0,6,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1990,0.0,290,55.0,116.0,0.474,0.0,0.0,NaN,55.0,116.0,0.474,0.474,25.0,44.0,0.568,27.0,62.0,89.0,12.0,4.0,12.0,22.0,39.0,135.0,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2,0,PF,1,0,0,0
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19.0,22,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1990,19.0,1505,417.0,1009.0,0.413,24.0,100.0,0.240,393.0,909.0,0.432,0.425,84.0,98.0,0.857,34.0,87.0,121.0,206.0,55.0,4.0,110.0,149.0,942.0,12.2,0.448,0.099,0.097,1.9,6.0,3.8,19.2,1.5,0.1,9.5,27.2,-0.7,-0.3,-1.0,-0.031,-2.0,-3.0,-5.0,-1.1,0,PG,0,1,0,0
2,3,Mark Acres,C,28,ORL,68,0.0,19,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1990,0.0,1313,109.0,214.0,0.509,1.0,3.0,0.333,108.0,211.0,0.512,0.512,66.0,101.0,0.653,140.0,219.0,359.0,25.0,25.0,25.0,42.0,218.0,285.0,9.2,0.551,0.014,0.472,11.3,18.7,14.9,2.5,0.9,1.1,14.0,9.3,1.4,1.1,2.5,0.090,-2.8,-0.2,-3.0,-0.3,0,C,0,0,0,0
3,4,Michael Adams,PG,28,DEN,66,66.0,35,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1990,66.0,2346,560.0,1421.0,0.394,167.0,564.0,0.296,393.0,857.0,0.459,0.453,465.0,529.0,0.879,58.0,198.0,256.0,693.0,147.0,6.0,240.0,162.0,1752.0,22.3,0.530,0.397,0.372,2.1,8.8,5.2,39.4,2.6,0.1,12.7,28.5,5.8,0.4,6.3,0.128,6.0,-0.7,5.3,4.3,0,PG,0,1,0,0
4,5,Mark Aguirre,SF,31,DET,78,13.0,25,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1990,13.0,2006,420.0,909.0,0.462,24.0,78.0,0.308,396.0,831.0,0.477,0.475,240.0,317.0,0.757,134.0,240.0,374.0,139.0,47.0,20.0,128.0,209.0,1104.0,16.7,0.526,0.086,0.349,7.6,13.7,10.7,11.6,1.2,0.6,10.9,25.7,2.8,2.7,5.5,0.132,1.2,0.2,1.4,1.7,0,SF,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18872,535,Thaddeus Young,PF,34,TOR,54,9.0,14,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2022,9.0,795,108.0,198.0,0.545,6.0,34.0,0.176,102.0,164.0,0.622,0.561,18.0,26.0,0.692,71.0,95.0,166.0,75.0,54.0,5.0,42.0,88.0,240.0,14.1,0.573,0.172,0.131,9.4,14.6,11.8,12.9,3.4,0.6,16.7,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,0,PF,1,0,0,0
18873,536,Trae Young,PG,24,ATL,73,73.0,34,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2022,73.0,2541,597.0,1390.0,0.429,154.0,460.0,0.335,443.0,930.0,0.476,0.485,566.0,639.0,0.886,56.0,161.0,217.0,741.0,80.0,9.0,300.0,104.0,1914.0,22.0,0.573,0.331,0.460,2.4,7.0,4.7,42.5,1.5,0.3,15.2,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,0,PG,0,1,0,0
18874,537,Omer Yurtseven,C,24,MIA,9,0.0,9,1.8,3.0,0.593,0.3,0.8,0.429,1.4,2.2,0.650,0.648,0.6,0.7,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,2022,0.0,83,16.0,27.0,0.593,3.0,7.0,0.429,13.0,20.0,0.650,0.648,5.0,6.0,0.833,8.0,15.0,23.0,2.0,2.0,2.0,4.0,16.0,40.0,16.7,0.675,0.259,0.222,10.9,21.9,16.2,3.9,1.2,2.5,11.9,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,0,C,0,0,0,0
18875,538,Cody Zeller,C

### Addressing Players With Mid-Season Transfers

In [28]:
# Keep Total Row
tot_rows = stats2[stats2.Tm == 'TOT'] # Only total rows
non_tot_rows = stats2.drop_duplicates(subset=['Player', 'Year'], keep=False) # Gets rid of all duplicate names
stats3 = pd.concat([non_tot_rows, tot_rows], axis=0).sort_values(by=['Year', 'Rk'])
stats3.head(10)

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,GT1_Pos,Pos_5,PF,PG,SF,SG
0,1,Alaa Abdelnaby,PF,22,POR,43,0.0,6,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1990,0.0,290,55.0,116.0,0.474,0.0,0.0,NaN,55.0,116.0,0.474,0.474,25.0,44.0,0.568,27.0,62.0,89.0,12.0,4.0,12.0,22.0,39.0,135.0,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2,0,PF,1,0,0,0
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19.0,22,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1990,19.0,1505,417.0,1009.0,0.413,24.0,100.0,0.240,393.0,909.0,0.432,0.425,84.0,98.0,0.857,34.0,87.0,121.0,206.0,55.0,4.0,110.0,149.0,942.0,12.2,0.448,0.099,0.097,1.9,6.0,3.8,19.2,1.5,0.1,9.5,27.2,-0.7,-0.3,-1.0,-0.031,-2.0,-3.0,-5.0,-1.1,0,PG,0,1,0,0
2,3,Mark Acres,C,28,ORL,68,0.0,19,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1990,0.0,1313,109.0,214.0,0.509,1.0,3.0,0.333,108.0,211.0,0.512,0.512,66.0,101.0,0.653,140.0,219.0,359.0,25.0,25.0,25.0,42.0,218.0,285.0,9.2,0.551,0.014,0.472,11.3,18.7,14.9,2.5,0.9,1.1,14.0,9.3,1.4,1.1,2.5,0.090,-2.8,-0.2,-3.0,-0.3,0,C,0,0,0,0
3,4,Michael Adams,PG,28,DEN,66,66.0,35,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1990,66.0,2346,560.0,1421.0,0.394,167.0,564.0,0.296,393.0,857.0,0.459,0.453,465.0,529.0,0.879,58.0,198.0,256.0,693.0,147.0,6.0,240.0,162.0,1752.0,22.3,0.530,0.397,0.372,2.1,8.8,5.2,39.4,2.6,0.1,12.7,28.5,5.8,0.4,6.3,0.128,6.0,-0.7,5.3,4.3,0,PG,0,1,0,0
4,5,Mark Aguirre,SF,31,DET,78,13.0,25,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1990,13.0,2006,420.0,909.0,0.462,24.0,78.0,0.308,396.0,831.0,0.477,0.475,240.0,317.0,0.757,134.0,240.0,374.0,139.0,47.0,20.0,128.0,209.0,1104.0,16.7,0.526,0.086,0.349,7.6,13.7,10.7,11.6,1.2,0.6,10.9,25.7,2.8,2.7,5.5,0.132,1.2,0.2,1.4,1.7,0,SF,0,0,1,0
5,6,Danny Ainge,SG,31,POR,80,0.0,21,4.2,8.9,0.472,1.3,3.1,0.406,2.9,5.8,0.508,0.543,1.4,1.7,0.826,0.6,2.0,2.6,3.6,0.8,0.2,1.3,2.4,11.1,1990,0.0,1710,337.0,714.0,0.472,102.0,251.0,0.406,235.0,463.0,0.508,0.543,114.0,138.0,0.826,45.0,160.0,205.0,285.0,63.0,13.0,100.0,195.0,890.0,17.0,0.574,0.352,0.193,2.9,10.2,6.6,23.8,1.8,0.5,11.4,20.8,4.1,2.1,6.2,0.175,2.8,0.9,3.7,2.5,0,SG,0,0,0,1
6,7,Mark Alarie,PF,27,WSB,42,1.0,14,2.4,5.4,0.440,0.1,0.5,0.238,2.2,4.9,0.461,0.451,1.0,1.1,0.854,1.0,1.8,2.8,1.1,0.4,0.2,1.0,2.1,5.8,1990,1.0,587,99.0,225.0,0.440,5.0,21.0,0.238,94.0,204.0,0.461,0.451,41.0,48.0,0.854,41.0,76.0,117.0,45.0,15.0,8.0,40.0,88.0,244.0,11.3,0.496,0.093,0.213,7.6,14.2,10.9,11.2,1.3,0.8,14.0,20.3,0.1,0.6,0.6,0.050,-3.0,-0.9,-3.8,-0.3,0,PF,1,0,0,0
7,8,Steve Alford,PG,26,DAL,34,0.0,6,1.7,3.4,0.504,0.2,0.7,0.304,1.5,2.8,0.553,0.534,0.8,0.9,0.839,0.3,0.4,0.7,0.6,0.2,0.0,0.5,0.3,4.4,1990,0.0,236,59.0,117.0,0.504,7.0,23.0,0.304,52.0,94.0,0.553,0.534,26.0,31.0,0.839,10.0,14.0,24.0,22.0,8.0,1.0,16.0,11.0,151.0,20.5,0.578,0.197,0.265,4.8,6.8,5.8,16.4,1.7,0.3,10.9,27.5,0.5,0.1,0.6,0.121,3.5,-1.4,2.1,0.2,0,PG,0,1,0,0
8,9,Greg Anderson,PF,26,TOT,68,2.0,13,1.7,4.0,0.430,0.0,0.0,0.000,1.7,4.0,0.431,0.430,0.9,1.7,0.522,1.4,3.3,4.7,0.2,0.5,0.7,1.2,2.1,4.3,1990,2.0,924,116.0,270.0,0.430,0.0,1.0,0.000,116.0,269.0,0.431,0.430,60.0,115.0,0.522,97.0,221.0,318.0,16.0,35.0,45.0,84.0,140.0,292.0,9.1,0.455,0.004,0.426,10.0,25.9,17.3,2.1,1.7,2.7,20.8,16.3,-1.2,0.9,-0.3,-0.017,-5.1,-1.0,-6.1,-1.0

In [29]:
stats2_dups = stats2[stats2.duplicated(subset=['Player', 'Year'], keep=False)] # Duplicates only
dups = stats2_dups[stats2_dups['Tm'] != 'TOT'] # Duplicates without TOT row
dups.head(10)

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,GT1_Pos,Pos_5,PF,PG,SF,SG
9,9,Greg Anderson,PF,26,MIL,26,0.0,9,1.0,2.8,0.370,0.0,0.0,0.000,1.0,2.8,0.375,0.370,0.6,1.1,0.571,1.0,1.9,2.9,0.1,0.3,0.3,0.8,1.1,2.7,1990,0.0,247,27.0,73.0,0.370,0.0,1.0,0.000,27.0,72.0,0.375,0.370,16.0,28.0,0.571,26.0,49.0,75.0,3.0,8.0,9.0,22.0,29.0,70.0,7.5,0.410,0.014,0.384,12.3,24.1,18.1,1.7,1.6,2.3,20.5,18.6,-0.5,0.4,-0.1,-0.027,-5.8,-2.1,-7.9,-0.4,0,PF,1,0,0,0
10,9,Greg Anderson,PF,26,NJN,1,0.0,18,4.0,4.0,1.000,0.0,0.0,NaN,4.0,4.0,1.000,1.000,0.0,0.0,NaN,4.0,2.0,6.0,1.0,2.0,0.0,1.0,4.0,8.0,1990,0.0,18,4.0,4.0,1.000,0.0,0.0,NaN,4.0,4.0,1.000,1.000,0.0,0.0,NaN,4.0,2.0,6.0,1.0,2.0,0.0,1.0,4.0,8.0,26.4,1.000,0.000,0.000,22.4,12.1,17.5,9.1,5.4,0.0,20.0,11.2,0.1,0.0,0.1,0.300,5.1,4.5,9.6,0.1,0,PF,1,0,0,0
11,9,Greg Anderson,PF,26,DEN,41,2.0,16,2.1,4.7,0.440,0.0,0.0,NaN,2.1,4.7,0.440,0.440,1.1,2.1,0.506,1.6,4.1,5.8,0.3,0.6,0.9,1.5,2.6,5.2,1990,2.0,659,85.0,193.0,0.440,0.0,0.0,NaN,85.0,193.0,0.440,0.440,44.0,87.0,0.506,67.0,170.0,237.0,12.0,25.0,36.0,61.0,107.0,214.0,9.2,0.463,0.000,0.451,8.7,27.0,17.0,2.1,1.6,2.9,20.9,15.6,-0.8,0.5,-0.3,-0.022,-5.2,-0.7,-5.9,-0.7,0,PF,1,0,0,0
27,24,Kenny Battle,SG,26,PHO,16,4.0,16,2.4,5.4,0.442,0.0,0.1,0.000,2.4,5.3,0.452,0.442,1.3,1.8,0.690,1.3,2.0,3.3,0.9,1.2,0.4,1.1,1.6,6.0,1990,4.0,263,38.0,86.0,0.442,0.0,2.0,0.000,38.0,84.0,0.452,0.442,20.0,29.0,0.690,21.0,32.0,53.0,15.0,19.0,6.0,17.0,25.0,96.0,12.8,0.486,0.023,0.337,9.2,12.7,11.0,7.5,3.4,1.3,14.7,17.9,0.0,0.4,0.4,0.081,-1.5,1.2,-0.3,0.1,0,SG,0,0,0,1
28,24,Kenny Battle,SG,26,DEN,40,4.0,17,2.4,4.9,0.485,0.1,0.6,0.136,2.3,4.4,0.529,0.492,1.3,1.6,0.781,1.6,1.5,3.1,1.2,1.0,0.3,0.9,2.1,6.1,1990,4.0,682,95.0,196.0,0.485,3.0,22.0,0.136,92.0,174.0,0.529,0.492,50.0,64.0,0.781,62.0,61.0,123.0,47.0,41.0,12.0,36.0,83.0,243.0,12.3,0.542,0.112,0.327,7.8,9.4,8.5,8.1,2.5,0.9,13.8,13.4,0.7,0.2,0.9,0.063,-1.2,0.3,-0.9,0.2,0,SG,0,0,0,1
31,26,Benoit Benjamin,C,26,LAC,39,38.0,34,5.9,11.9,0.492,0.0,0.0,NaN,5.9,11.9,0.492,0.492,3.2,4.3,0.728,2.4,9.6,12.0,1.9,0.7,2.3,3.5,2.8,14.9,1990,38.0,1337,229.0,465.0,0.492,0.0,0.0,NaN,229.0,465.0,0.492,0.492,123.0,169.0,0.728,95.0,374.0,469.0,74.0,26.0,91.0,138.0,110.0,581.0,15.2,0.539,0.000,0.363,7.5,30.7,18.9,8.1,0.9,4.1,20.4,20.6,-0.5,2.4,1.9,0.068,-2.1,1.1,-1.0,0.3,0,C,0,0,0,0
32,26,Benoit Benjamin,C,26,SEA,31,27.0,29,5.1,10.1,0.502,0.0,0.0,NaN,5.1,10.1,0.502,0.502,2.8,4.1,0.690,2.0,6.2,8.2,1.5,0.9,1.7,3.1,2.4,12.9,1990,27.0,899,157.0,313.0,0.502,0.0,0.0,NaN,157.0,313.0,0.502,0.502,87.0,126.0,0.690,62.0,192.0,254.0,45.0,28.0,54.0,97.0,74.0,401.0,15.1,0.544,0.000,0.403,8.1,25.9,16.9,7.1,1.6,3.9,20.8,21.8,-0.2,1.3,1.1,0.059,-1.6,0.3,-1.2,0.2,0,C,0,0,0,0
49,42,Tony Brown,SF,30,LAL,7,0.0,3,0.3,0.4,0.667,0.1,0.1,1.000,0.1,0.3,0.500,0.833,0.0,0.0,NaN,0.0,0.6,0.6,0.4,0.0,0.0,0.6,1.1,0.7,1990,0.0,27,2.0,3.0,0.667,1.0,1.0,1.000,1.0,2.0,0.500,0.833,0.0,0.0,NaN,0.0,4.0,4.0,3.0,0.0,0.0,4.0,8.0,5.0,-0.4,0.833,0.333,0.000,0.0,16.5,8.6,14.5,0.0,0.0,57.1,11.3,-0.1,0.0,0.0,-0.043,-8.5,0.7,-7.7,0.0,0,SF,0,0,1,0
50,42,Tony Brown,SG,30,UTA,23,0.0,11,1.2,3.3,0.364,0.1,0.5,0.182,1.1,2.9,0.394,0.377,0.9,1.0,0.870,1.0,0.7,1.7,0.6,0.2,0.0,0.5,1.7,3.4,1990,0.0,267,28.0,77.0,0.364,2.0,11.0,0.182,26.0,66.0,0.394,0.377,20.0,23.0,0.870,24.0,15.0,39.0,13.0,4.0,0.0,12.0,39.0,78.0,6.5,0.448,0.143,0.299,11.3,6.2,8.6,6.9,0.8,0.0,12.1,16.4,0.0,0.2,0.2,0.033,-4.2,-1.9,-6.1,-0.3,0,SG,0,0,0,1
76,67,Lester Conner,PG,31,NJN,35,2.0,14,1.7,3.2,0.523,0.0,0.1,0.000,1.7,3.1,0.532,0.523,0.8,1.2,0.690

In [30]:
# Take the Team where the player played most games
idx = dups.groupby(['Player', 'Year'])['G'].idxmax()
team_mrg = dups.loc[idx].filter(items = ['Rk', 'Player', 'Tm', 'Year'])
team_mrg['Midseason_Trade'] = 1
team_mrg.rename(columns = {'Tm': 'Tm2'}, inplace=True)
team_mrg

,Rk,Player,Tm2,Year,Midseason_Trade
3022,147,A.C. Green,DAL,1996,1
18561,284,A.J. Lawson,DAL,2022,1
13305,373,A.J. Price,CLE,2014,1
10566,64,Aaron Brooks,HOU,2010,1
11723,60,Aaron Brooks,SAC,2012,1
...,...,...,...,...,...
14681,23,Ömer Aşık,NOP,2017,1
8442,211,Šarūnas Jasikevičius,IND,2006,1
3863,381,Žan Tabak,TOR,1997,1
7147,68,Žarko Čabarkapa,GSW,2004,1


In [31]:
# Replace Tm With The Team Player Had Most Games With
stats4 = stats3.merge(team_mrg, how = 'left', on = ['Rk', 'Player', 'Year'])
stats4.head(10)

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,GT1_Pos,Pos_5,PF,PG,SF,SG,Tm2,Midseason_Trade
0,1,Alaa Abdelnaby,PF,22,POR,43,0.0,6,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1990,0.0,290,55.0,116.0,0.474,0.0,0.0,NaN,55.0,116.0,0.474,0.474,25.0,44.0,0.568,27.0,62.0,89.0,12.0,4.0,12.0,22.0,39.0,135.0,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2,0,PF,1,0,0,0,NaN,NaN
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19.0,22,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1990,19.0,1505,417.0,1009.0,0.413,24.0,100.0,0.240,393.0,909.0,0.432,0.425,84.0,98.0,0.857,34.0,87.0,121.0,206.0,55.0,4.0,110.0,149.0,942.0,12.2,0.448,0.099,0.097,1.9,6.0,3.8,19.2,1.5,0.1,9.5,27.2,-0.7,-0.3,-1.0,-0.031,-2.0,-3.0,-5.0,-1.1,0,PG,0,1,0,0,NaN,NaN
2,3,Mark Acres,C,28,ORL,68,0.0,19,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1990,0.0,1313,109.0,214.0,0.509,1.0,3.0,0.333,108.0,211.0,0.512,0.512,66.0,101.0,0.653,140.0,219.0,359.0,25.0,25.0,25.0,42.0,218.0,285.0,9.2,0.551,0.014,0.472,11.3,18.7,14.9,2.5,0.9,1.1,14.0,9.3,1.4,1.1,2.5,0.090,-2.8,-0.2,-3.0,-0.3,0,C,0,0,0,0,NaN,NaN
3,4,Michael Adams,PG,28,DEN,66,66.0,35,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1990,66.0,2346,560.0,1421.0,0.394,167.0,564.0,0.296,393.0,857.0,0.459,0.453,465.0,529.0,0.879,58.0,198.0,256.0,693.0,147.0,6.0,240.0,162.0,1752.0,22.3,0.530,0.397,0.372,2.1,8.8,5.2,39.4,2.6,0.1,12.7,28.5,5.8,0.4,6.3,0.128,6.0,-0.7,5.3,4.3,0,PG,0,1,0,0,NaN,NaN
4,5,Mark Aguirre,SF,31,DET,78,13.0,25,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1990,13.0,2006,420.0,909.0,0.462,24.0,78.0,0.308,396.0,831.0,0.477,0.475,240.0,317.0,0.757,134.0,240.0,374.0,139.0,47.0,20.0,128.0,209.0,1104.0,16.7,0.526,0.086,0.349,7.6,13.7,10.7,11.6,1.2,0.6,10.9,25.7,2.8,2.7,5.5,0.132,1.2,0.2,1.4,1.7,0,SF,0,0,1,0,NaN,NaN
5,6,Danny Ainge,SG,31,POR,80,0.0,21,4.2,8.9,0.472,1.3,3.1,0.406,2.9,5.8,0.508,0.543,1.4,1.7,0.826,0.6,2.0,2.6,3.6,0.8,0.2,1.3,2.4,11.1,1990,0.0,1710,337.0,714.0,0.472,102.0,251.0,0.406,235.0,463.0,0.508,0.543,114.0,138.0,0.826,45.0,160.0,205.0,285.0,63.0,13.0,100.0,195.0,890.0,17.0,0.574,0.352,0.193,2.9,10.2,6.6,23.8,1.8,0.5,11.4,20.8,4.1,2.1,6.2,0.175,2.8,0.9,3.7,2.5,0,SG,0,0,0,1,NaN,NaN
6,7,Mark Alarie,PF,27,WSB,42,1.0,14,2.4,5.4,0.440,0.1,0.5,0.238,2.2,4.9,0.461,0.451,1.0,1.1,0.854,1.0,1.8,2.8,1.1,0.4,0.2,1.0,2.1,5.8,1990,1.0,587,99.0,225.0,0.440,5.0,21.0,0.238,94.0,204.0,0.461,0.451,41.0,48.0,0.854,41.0,76.0,117.0,45.0,15.0,8.0,40.0,88.0,244.0,11.3,0.496,0.093,0.213,7.6,14.2,10.9,11.2,1.3,0.8,14.0,20.3,0.1,0.6,0.6,0.050,-3.0,-0.9,-3.8,-0.3,0,PF,1,0,0,0,NaN,NaN
7,8,Steve Alford,PG,26,DAL,34,0.0,6,1.7,3.4,0.504,0.2,0.7,0.304,1.5,2.8,0.553,0.534,0.8,0.9,0.839,0.3,0.4,0.7,0.6,0.2,0.0,0.5,0.3,4.4,1990,0.0,236,59.0,117.0,0.504,7.0,23.0,0.304,52.0,94.0,0.553,0.534,26.0,31.0,0.839,10.0,14.0,24.0,22.0,8.0,1.0,16.0,11.0,151.0,20.5,0.578,0.197,0.265,4.8,6.8,5.8,16.4,1.7,0.3,10.9,27.5,0.5,0.1,0.6,0.121,3.5,-1.4,2.1,0.2,0,PG,0,1,0,0,NaN,NaN
8,9,Greg Anderson,PF,26,TOT,68,2.0,13,1.7,4.0,0.430,0.0,0.0,0.000,1.7,4.0,0.431,0.430,0.9,1.7,0.522,1.4,3.3,4.7,0.2,0.5,0.7,1.2,2.1,4.3,1990,2.0,924,116.0,270.0,0.430,0.0,1.0,0.000,116.0,269.0,0.431,0.430,60.0,115.0,0.522,97.0,221.0,318.0,16.0,35.0,45.0,84.0,140.0,292.0,9.1,0.455,0.004

In [32]:
# Replace TOT in 'Tm' with corresponding values in 'Tm2'
stats4.loc[stats4['Tm'] == 'TOT', 'Tm'] = stats4.loc[stats4['Tm'] == 'TOT', 'Tm2']

# Drop Tm2 
stats4.drop(columns=['Tm2'], inplace=True)
stats4.head(10)

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,GT1_Pos,Pos_5,PF,PG,SF,SG,Midseason_Trade
0,1,Alaa Abdelnaby,PF,22,POR,43,0.0,6,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1990,0.0,290,55.0,116.0,0.474,0.0,0.0,NaN,55.0,116.0,0.474,0.474,25.0,44.0,0.568,27.0,62.0,89.0,12.0,4.0,12.0,22.0,39.0,135.0,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2,0,PF,1,0,0,0,NaN
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19.0,22,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1990,19.0,1505,417.0,1009.0,0.413,24.0,100.0,0.240,393.0,909.0,0.432,0.425,84.0,98.0,0.857,34.0,87.0,121.0,206.0,55.0,4.0,110.0,149.0,942.0,12.2,0.448,0.099,0.097,1.9,6.0,3.8,19.2,1.5,0.1,9.5,27.2,-0.7,-0.3,-1.0,-0.031,-2.0,-3.0,-5.0,-1.1,0,PG,0,1,0,0,NaN
2,3,Mark Acres,C,28,ORL,68,0.0,19,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1990,0.0,1313,109.0,214.0,0.509,1.0,3.0,0.333,108.0,211.0,0.512,0.512,66.0,101.0,0.653,140.0,219.0,359.0,25.0,25.0,25.0,42.0,218.0,285.0,9.2,0.551,0.014,0.472,11.3,18.7,14.9,2.5,0.9,1.1,14.0,9.3,1.4,1.1,2.5,0.090,-2.8,-0.2,-3.0,-0.3,0,C,0,0,0,0,NaN
3,4,Michael Adams,PG,28,DEN,66,66.0,35,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1990,66.0,2346,560.0,1421.0,0.394,167.0,564.0,0.296,393.0,857.0,0.459,0.453,465.0,529.0,0.879,58.0,198.0,256.0,693.0,147.0,6.0,240.0,162.0,1752.0,22.3,0.530,0.397,0.372,2.1,8.8,5.2,39.4,2.6,0.1,12.7,28.5,5.8,0.4,6.3,0.128,6.0,-0.7,5.3,4.3,0,PG,0,1,0,0,NaN
4,5,Mark Aguirre,SF,31,DET,78,13.0,25,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1990,13.0,2006,420.0,909.0,0.462,24.0,78.0,0.308,396.0,831.0,0.477,0.475,240.0,317.0,0.757,134.0,240.0,374.0,139.0,47.0,20.0,128.0,209.0,1104.0,16.7,0.526,0.086,0.349,7.6,13.7,10.7,11.6,1.2,0.6,10.9,25.7,2.8,2.7,5.5,0.132,1.2,0.2,1.4,1.7,0,SF,0,0,1,0,NaN
5,6,Danny Ainge,SG,31,POR,80,0.0,21,4.2,8.9,0.472,1.3,3.1,0.406,2.9,5.8,0.508,0.543,1.4,1.7,0.826,0.6,2.0,2.6,3.6,0.8,0.2,1.3,2.4,11.1,1990,0.0,1710,337.0,714.0,0.472,102.0,251.0,0.406,235.0,463.0,0.508,0.543,114.0,138.0,0.826,45.0,160.0,205.0,285.0,63.0,13.0,100.0,195.0,890.0,17.0,0.574,0.352,0.193,2.9,10.2,6.6,23.8,1.8,0.5,11.4,20.8,4.1,2.1,6.2,0.175,2.8,0.9,3.7,2.5,0,SG,0,0,0,1,NaN
6,7,Mark Alarie,PF,27,WSB,42,1.0,14,2.4,5.4,0.440,0.1,0.5,0.238,2.2,4.9,0.461,0.451,1.0,1.1,0.854,1.0,1.8,2.8,1.1,0.4,0.2,1.0,2.1,5.8,1990,1.0,587,99.0,225.0,0.440,5.0,21.0,0.238,94.0,204.0,0.461,0.451,41.0,48.0,0.854,41.0,76.0,117.0,45.0,15.0,8.0,40.0,88.0,244.0,11.3,0.496,0.093,0.213,7.6,14.2,10.9,11.2,1.3,0.8,14.0,20.3,0.1,0.6,0.6,0.050,-3.0,-0.9,-3.8,-0.3,0,PF,1,0,0,0,NaN
7,8,Steve Alford,PG,26,DAL,34,0.0,6,1.7,3.4,0.504,0.2,0.7,0.304,1.5,2.8,0.553,0.534,0.8,0.9,0.839,0.3,0.4,0.7,0.6,0.2,0.0,0.5,0.3,4.4,1990,0.0,236,59.0,117.0,0.504,7.0,23.0,0.304,52.0,94.0,0.553,0.534,26.0,31.0,0.839,10.0,14.0,24.0,22.0,8.0,1.0,16.0,11.0,151.0,20.5,0.578,0.197,0.265,4.8,6.8,5.8,16.4,1.7,0.3,10.9,27.5,0.5,0.1,0.6,0.121,3.5,-1.4,2.1,0.2,0,PG,0,1,0,0,NaN
8,9,Greg Anderson,PF,26,DEN,68,2.0,13,1.7,4.0,0.430,0.0,0.0,0.000,1.7,4.0,0.431,0.430,0.9,1.7,0.522,1.4,3.3,4.7,0.2,0.5,0.7,1.2,2.1,4.3,1990,2.0,924,116.0,270.0,0.430,0.0,1.0,0.000,116.0,269.0,0.431,0.430,60.0,115.0,0.522,97.0,221.0,318.0,16.0,35.0,45.0,84.0,140.0,292.0,9.1,0.455,0.004,0.426,10.0,25.9,17.3,2.1,1.7,2.7,20

In [33]:
stats4['Midseason_Trade'] = np.where(stats4['Midseason_Trade'].isnull(), 0, 1)
stats4

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,GT1_Pos,Pos_5,PF,PG,SF,SG,Midseason_Trade
0,1,Alaa Abdelnaby,PF,22,POR,43,0.0,6,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1990,0.0,290,55.0,116.0,0.474,0.0,0.0,NaN,55.0,116.0,0.474,0.474,25.0,44.0,0.568,27.0,62.0,89.0,12.0,4.0,12.0,22.0,39.0,135.0,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2,0,PF,1,0,0,0,0
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19.0,22,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1990,19.0,1505,417.0,1009.0,0.413,24.0,100.0,0.240,393.0,909.0,0.432,0.425,84.0,98.0,0.857,34.0,87.0,121.0,206.0,55.0,4.0,110.0,149.0,942.0,12.2,0.448,0.099,0.097,1.9,6.0,3.8,19.2,1.5,0.1,9.5,27.2,-0.7,-0.3,-1.0,-0.031,-2.0,-3.0,-5.0,-1.1,0,PG,0,1,0,0,0
2,3,Mark Acres,C,28,ORL,68,0.0,19,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1990,0.0,1313,109.0,214.0,0.509,1.0,3.0,0.333,108.0,211.0,0.512,0.512,66.0,101.0,0.653,140.0,219.0,359.0,25.0,25.0,25.0,42.0,218.0,285.0,9.2,0.551,0.014,0.472,11.3,18.7,14.9,2.5,0.9,1.1,14.0,9.3,1.4,1.1,2.5,0.090,-2.8,-0.2,-3.0,-0.3,0,C,0,0,0,0,0
3,4,Michael Adams,PG,28,DEN,66,66.0,35,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1990,66.0,2346,560.0,1421.0,0.394,167.0,564.0,0.296,393.0,857.0,0.459,0.453,465.0,529.0,0.879,58.0,198.0,256.0,693.0,147.0,6.0,240.0,162.0,1752.0,22.3,0.530,0.397,0.372,2.1,8.8,5.2,39.4,2.6,0.1,12.7,28.5,5.8,0.4,6.3,0.128,6.0,-0.7,5.3,4.3,0,PG,0,1,0,0,0
4,5,Mark Aguirre,SF,31,DET,78,13.0,25,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1990,13.0,2006,420.0,909.0,0.462,24.0,78.0,0.308,396.0,831.0,0.477,0.475,240.0,317.0,0.757,134.0,240.0,374.0,139.0,47.0,20.0,128.0,209.0,1104.0,16.7,0.526,0.086,0.349,7.6,13.7,10.7,11.6,1.2,0.6,10.9,25.7,2.8,2.7,5.5,0.132,1.2,0.2,1.4,1.7,0,SF,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15231,535,Thaddeus Young,PF,34,TOR,54,9.0,14,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2022,9.0,795,108.0,198.0,0.545,6.0,34.0,0.176,102.0,164.0,0.622,0.561,18.0,26.0,0.692,71.0,95.0,166.0,75.0,54.0,5.0,42.0,88.0,240.0,14.1,0.573,0.172,0.131,9.4,14.6,11.8,12.9,3.4,0.6,16.7,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,0,PF,1,0,0,0,0
15232,536,Trae Young,PG,24,ATL,73,73.0,34,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2022,73.0,2541,597.0,1390.0,0.429,154.0,460.0,0.335,443.0,930.0,0.476,0.485,566.0,639.0,0.886,56.0,161.0,217.0,741.0,80.0,9.0,300.0,104.0,1914.0,22.0,0.573,0.331,0.460,2.4,7.0,4.7,42.5,1.5,0.3,15.2,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,0,PG,0,1,0,0,0
15233,537,Omer Yurtseven,C,24,MIA,9,0.0,9,1.8,3.0,0.593,0.3,0.8,0.429,1.4,2.2,0.650,0.648,0.6,0.7,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,2022,0.0,83,16.0,27.0,0.593,3.0,7.0,0.429,13.0,20.0,0.650,0.648,5.0,6.0,0.833,8.0,15.0,23.0,2.0,2.0,2.0,4.0,16.0,40.0,16.7,0.675,0.259,0.222,10.9,21.9,16.2,3.9,1.2,2.5,11.9,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,0

In [34]:
# Check that Midseason_Trade is 0s and 1s
stats4.Midseason_Trade.value_counts()

0    13484
1     1752
Name: Midseason_Trade, dtype: int64

In [35]:
# Check that there is no more TOT
chk = stats4['Tm'].value_counts()
if 'TOT' in chk.index:
    print("'TOT' present")
else:
    print("'TOT' is not present")

'TOT' is not present


### Merge All-NBA Winner Indicators

In [36]:
allnba = pd.read_csv('../data/all_nba_teams.csv')
allnba.head()

,# Tm,Pos,Player,Age,Tm,Pts Won,Pts Max,Share,1st Tm,2nd Tm,3rd Tm,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1st,G,Michael Jordan,27,CHI,476,480,0.992,94.0,NaN,NaN,82,37.0,31.5,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321,1990
1,1st,G,Magic Johnson,31,LAL,472,480,0.983,93.0,NaN,NaN,79,37.1,19.4,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251,1990
2,1st,F,Karl Malone,27,UTA,464,480,0.967,88.0,NaN,NaN,82,40.3,29.0,11.8,3.3,1.1,1.0,0.527,0.286,0.770,15.5,0.225,1990
3,1st,C,David Robinson,25,SAS,458,480,0.954,87.0,NaN,NaN,82,37.7,25.6,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264,1990
4,1st,F,Charles Barkley,27,PHI,452,480,0.942,82.0,NaN,NaN,67,37.3,27.6,10.1,4.2,1.6,0.5,0.570,0.284,0.722,13.4,0.258,1990


In [37]:
allnba2 = allnba.filter(items=['# Tm', 'Player', 'Pts Won', 'Pts Max', 'Share', 'Year'])

In [38]:
allnba2.count()

# Tm       1354
Player     1354
Pts Won    1354
Pts Max    1354
Share      1354
Year       1354
dtype: int64

In [39]:
def placement(tm):
    if tm in ('1st', '1T'):
        return 1
    elif tm in ('2nd', '2T'):
        return 2
    elif tm in ('3rd', '3T'):
        return 3
    else:
        return 0

In [40]:
allnba2['All_NBA_Winner'] = allnba2['# Tm'].apply(placement)
allnba2.rename(columns = {'# Tm' : 'All_NBA_Team'}, inplace=True)
allnba2.head()

,All_NBA_Team,Player,Pts Won,Pts Max,Share,Year,All_NBA_Winner
0,1st,Michael Jordan,476,480,0.992,1990,1
1,1st,Magic Johnson,472,480,0.983,1990,1
2,1st,Karl Malone,464,480,0.967,1990,1
3,1st,David Robinson,458,480,0.954,1990,1
4,1st,Charles Barkley,452,480,0.942,1990,1


In [41]:
stats4['Player'] = stats4['Player'].str.replace('*', '')
allnba2['Player'] = allnba2['Player'].str.replace('*', '')

In [42]:
import unicodedata
def rmv_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

allnba2['Player'] = allnba2.Player.apply(rmv_accents)
stats4['Player'] = stats4.Player.apply(rmv_accents)

In [43]:
stats5 = stats4.applymap(lambda x: x.strip() if isinstance(x, str) else x)
stats5 = stats5.merge(allnba2, how = 'left', on = ['Player', 'Year'])

In [44]:
stats5['All_NBA_Team'] = stats5['All_NBA_Team'].fillna(0)
stats5['Pts Won'] = stats5['Pts Won'].fillna(0)
stats5['Pts Max'] = stats5['Pts Max'].fillna(0)
stats5['Share'] = stats5['Share'].fillna(0)
stats5['All_NBA_Winner'] = stats5['All_NBA_Winner'].fillna(0)

### Assess Missing Values, Null Values, and Outliers

In [45]:
stats5.describe()

,Rk,Age,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,GT1_Pos,PF,PG,SF,SG,Midseason_Trade,Pts Won,Pts Max,Share,All_NBA_Winner
count,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15175.000000,15236.000000,15236.000000,13134.000000,15236.000000,15236.000000,15131.000000,15175.000000,15236.000000,15236.000000,14691.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15175.000000,15236.000000,15236.000000,13134.000000,15236.000000,15236.000000,15131.000000,15175.000000,15236.000000,15236.000000,14691.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15233.000000,15181.000000,15175.000000,15175.000000,15233.000000,15233.000000,15233.000000,15233.000000,15233.000000,15233.000000,15191.000000,15233.000000,15236.000000,15236.000000,15236.000000,15233.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000,15236.000000
mean,234.062287,26.674718,52.055001,25.316290,19.860265,3.103531,6.886794,0.439473,0.569940,1.635213,0.278360,2.533427,5.251405,0.469813,0.477868,1.509917,2.017032,0.727184,0.982187,2.590135,3.570786,1.857102,0.653676,0.414302,1.194605,1.860462,8.284543,2006.925243,25.316290,1223.906340,192.198215,420.681478,0.439473,34.804936,97.869126,0.278360,157.393279,322.812352,0.469813,0.477868,93.993043,124.413297,0.727184,58.835652,156.845563,215.681216,114.393345,39.523694,25.104096,71.731032,108.247309,513.194408,12.810668,0.514788,0.230664,0.301123,5.843380,14.317534,10.081908,13.055380,1.618788,1.605600,14.066474,18.788131,1.345727,1.248228,2.594992,0.073395,-1.490345,-0.186794,-1.677304,0.619585,0.015621,0.208257,0.193817,0.186335,0.207272,0.114991,11.125295,50.921502,0.019491,0.064978
std,138.080514,4.247572,25.194122,28.906809,10.081087,2.216693,4.657726,0.095175,0.699729,1.844348,0.160156,1.969593,3.885848,0.103293,0.098318,1.404211,1.773497,0.143413,0.840708,1.807471,2.514820,1.836253,0.456295,0.484182,0.805926,0.827884,6.037396,9.557783,28.906809,903.767754,175.080233,372.566967,0.095175,48.599927,128.536434,0.160156,153.492628,306.648774,0.103293,0.098318,105.211484,134.033572,0.143413,63.019326,142.892304,199.020705,135.208942,35.754452,34.674111,63.608499,75.028710,474.330241,6.269065,0.094247,0.215713,0.213894,4.937068,6.525686,4.993534,9.450223,1.018944,1.838564,6.318287,5.514614,2.049484,1.206165,2.966328,0.102077,4.077035,1.840518,5.085965,1.350236,0.124536,0.406075,0.395300,0.389390,0.405366,0.319021,64.476348,163.982469,0.112391,0.383930
min,1.000000,18.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1990.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-90.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.300000,-1.000000,-2.100000,-2.519000,-68.900000,-31.100000,-92.100000,-2.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [46]:
stats5.isnull().sum()[stats5.isnull().sum() > 0]

Tm              1
pg_FG%         61
pg_3P%       2102
pg_2P%        105
pg_eFG%        61
pg_FT%        545
tot_FG%        61
tot_3P%      2102
tot_2P%       105
tot_eFG%       61
tot_FT%       545
adv_PER         3
adv_TS%        55
adv_3PAr       61
adv_FTr        61
adv_ORB%        3
adv_DRB%        3
adv_TRB%        3
adv_AST%        3
adv_STL%        3
adv_BLK%        3
adv_TOV%       45
adv_USG%        3
adv_WS/48       3
dtype: int64

In [47]:
stats5[stats5['pg_FG%'].isnull()].G.value_counts()

1    38
2    15
3     5
4     2
5     1
Name: G, dtype: int64

##### <span style = 'color:mediumvioletred'> _All players who have missing FG% stats only played between 1-5 games - we can safely remove these players as they will not be chosen for the All-NBA team and would not help train our model will with so few game statistics._ </span>

In [48]:
stats5[stats5['pg_3P%'].isnull()].Pos_5.value_counts()

C     1222
PF     603
SF     131
SG      75
PG      71
Name: Pos_5, dtype: int64

In [49]:
stats5[stats5['pg_3P%'].isnull()].Pos_5.value_counts(normalize=True)

C     0.581351
PF    0.286870
SF    0.062322
SG    0.035680
PG    0.033777
Name: Pos_5, dtype: float64

In [50]:
miss_3p = stats5[stats5['pg_3P%'].isnull()]

In [51]:
miss_3p.groupby('Pos_5').agg({
    'G' : ['mean', 'median', 'min', 'max']})

G               
            mean median min max
Pos_5                          
C      38.914894   37.0   1  85
PF     32.001658   26.0   1  82
PG      6.295775    3.0   1  40
SF     19.274809    9.0   1  82
SG      6.013333    3.0   1  60

##### <span style = 'color:mediumvioletred'> _~60% of players who had missing 3P% were Centers and Power Forwards, who are not expected to attempt 3-point shots regularly or at all - these missing values check out / make sense. For positions where 3-point attempts are more typical - such as Shooting Guard, Point Guard, and Small Forward - these players likely did not play in enough games to build these stats. We will likely be dropping these players without a minimum number of games played during later phases of this cleanup._ </span>

In [52]:
print('Average Minutes Played: ' + str(miss_3p[(miss_3p.G >= 40) & (~miss_3p.Pos_5.isin(['PF', 'C']))].pg_MP.mean()))
print('Average Points: ' + str(miss_3p[(miss_3p.G >= 40) & (~miss_3p.Pos_5.isin(['PF', 'C']))].pg_PTS.mean()))
print('Average 3PA: ' + str(miss_3p[(miss_3p.G >= 40) & (~miss_3p.Pos_5.isin(['PF', 'C']))].pg_3PA.mean()))

Average Minutes Played: 15.37037037037037
Average Points: 5.48148148148148
Average 3PA: 0.0


##### <span style = 'color:mediumvioletred'> _~Among PGs, SGs, and SFs who played half the games of the season and still had NaN for their 3P%, these players played on average 15 minutes per game, but scored only ~5 points on average, and most importantly, attempted 0 3P shots. These players will likely be removed._ </span>

In [53]:
miss_2p = stats5[stats5['pg_2P%'].isnull()]
print('Average Minutes Played: ' + str(miss_2p[(miss_2p.G >= 40) & (~miss_2p.Pos_5.isin(['PF', 'C']))].pg_MP.mean()))
print('Average Points: ' + str(miss_2p[(miss_2p.G >= 40) & (~miss_2p.Pos_5.isin(['PF', 'C']))].pg_PTS.mean()))
print('Average 2PA: ' + str(miss_2p[(miss_2p.G >= 40) & (~miss_2p.Pos_5.isin(['PF', 'C']))].pg_2PA.mean()))

Average Minutes Played: nan
Average Points: nan
Average 2PA: nan


##### <span style = 'color:mediumvioletred'> _It is clear that many of the players with missing observations may be removed from our dataframe, as they have not generated ample stats to be useful for our model. Before going further with examining all individual missing values, we will establish some minimum criteria for inclusion in our dataset and all other players will be removed. We will then revisit missing/null values._ </span>

## II. Setting Minimum Criteria Using Domain Knowledge
##### To remove noise and unhelpful players from our data, I mined through various statistics websites (e.g., [StatMuse](www.statmuse.com), [RealGM](https://basketball.realgm.com/nba/stats), [Basketball-Reference](https://www.basketball-reference.com/players/)) to establish minimums based on historical data. 

Players^ will be filtered by the following historical lows:
- <u>Minimum Number of Games Played</u>: 41
    - Gus Johnson (1965-66 season) had 41 games played when selected as All-NBA team winner (2nd team)
- <u>Minimum Number of Minutes Played Per Game</u>: 26
    - Bob Cousy (1962-63 season) had 26 minutes per game played when selected as an All-NBA team winner (1st team)
- <u>Minimum Points Per Game</u>: 6.9
    - Ben Wallace (2002-2003 season) had 6.9 ppg when selected as an All-NBA team winner (2nd team)
- <u>Minimum Player Efficiency Ranking</u>: 11
    - Slater Martin (1957-58 & 1958-59 seasons) had a 9.3 PER when selected as an All-NBA team winner (2nd team)
    - We will cap our results at 11, which is close to the next lowest amount, as 9.3 seems anomalous and this data may be outdated based on current-day voting standards; 11 is still well below the league average of 15.
    
^ These criteria will not apply to players who had any amount of voter share towards an All-NBA team. There exist players who, while they did not win an All-NBA team spot, did receive votes for consideration. Voter Share is our target variable, therefore we will keep all nominees as well as winners. 

**Note:** On July 1, 2023, the new 7-year collective bargaining agreement (CBA) was signed and will run through 2029-30. Starting in the 2023-24 season, players must be on the floor for 20 minutes in at least 65 games to be eligible for honors including NBA MVP and All-NBA teams. These filtering criteria will be put in place for 2023 and onwards in our model.

Read the CBA [here](https://ak-static.cms.nba.com/wp-content/uploads/sites/4/2023/06/2023-NBA-Collective-Bargaining-Agreement.pdf) and see summaries [here](https://ak-static.cms.nba.com/wp-content/uploads/sites/4/2023/06/2023-CBA-Summary.pdf) and [here](https://theathletic.com/4421655/2023/04/17/cba-minimum-games-mvp-awards/).

### Games Played, Minutes, Points, and Player Efficiency

In [54]:
# Divide data into before and after 2023
stats5_b4_2023 = stats5[stats5.Year < 2023]
stats5_after_2023 = stats5[stats5.Year >= 2023]

# PRE-2023 --> at least 41 games played, 26 minutes per game, 6.9 ppg, and PER of 13
stats6_1 = stats5_b4_2023[(stats5_b4_2023.G >= 41) & (stats5_b4_2023.pg_MP >= 26) & (stats5_b4_2023.pg_PTS >= 6.9) & (stats5_b4_2023.adv_PER >= 11) & (stats5_b4_2023.All_NBA_Team == 0)]

# POST-2023 --> same as pre-2023 EXCEPT players must be on the floor for 20 minutes in at least 65 games, for a total of 1300 minutes minimum; however, there are protections for near-misses.
    # For up to 2 games, the player may only have played 15 minutes, so we'll do 1300 - ((20-15) * 2) = 1290 OR (63*20) + 2(15) = 1290
stats6_2 = stats5_after_2023[(stats5_after_2023.G >= 41) & (stats5_after_2023.tot_MP >= 1290) & (stats5_after_2023.pg_PTS >= 6.9) & (stats5_after_2023.adv_PER >= 11) & (stats5_after_2023.All_NBA_Team == 0)]

stats6_3 = stats5[stats5.All_NBA_Team != 0]

stats6 = pd.concat([stats6_1, stats6_2, stats6_3], axis=0)

In [55]:
print(f'Original Data: {stats5.shape}')
print(f'Filtered Data: {stats6.shape}')
print(f'All-NBA Data: {stats6_3.shape}')

Original Data: (15236, 87)
Filtered Data: (4353, 87)
All-NBA Data: (1353, 87)


In [56]:
stats6.groupby('Year').Rk.count()

Year
1990    124
1991    125
1992    126
1993    127
1994    129
1995    136
1996    138
1997    141
1998    106
1999    121
2000    128
2001    127
2002    131
2003    131
2004    137
2005    135
2006    145
2007    134
2008    142
2009    139
2010    136
2011    130
2012    135
2013    143
2014    132
2015    129
2016    127
2017    125
2018    136
2019    130
2020    129
2021    136
2022    143
Name: Rk, dtype: int64

### Re-Assess Missing Values

In [57]:
# Re-Check Missing Values
stats6.isnull().sum()[stats6.isnull().sum() > 0]

pg_3P%     156
tot_3P%    156
dtype: int64

In [58]:
stats6[stats6['pg_3P%'].isnull()].Pos_5.value_counts()

C     112
PF     41
SF      3
Name: Pos_5, dtype: int64

In [59]:
stats6[stats6['pg_3P%'].isnull()].Pos_5.value_counts(normalize=True)

C     0.717949
PF    0.262821
SF    0.019231
Name: Pos_5, dtype: float64

##### <span style = 'color:mediumvioletred'> _After setting minimum selection, there are no more unexpected missings in the data. There are 156 per-game and total 3P%s missing, however this is explained by position. Centers are not expected to attempt 3-pointers, and most power forwards do not attempt any as well. There are only 3 players who could potentially be taking 3-pointers, but were not._ </span>

In [60]:
# Will not be feature engineering 'Years_in_league' variable, as there are several anomalous names to account for (e.g., accented or special character names which are not matching between dataframes), 
    # as well as undrafted players who will not appear in draft_year.csv. The information gain from including this variable may not outweigh the processing/manipulation time.
#draft = pd.read_csv('../data/draft_year.csv')
#draft = draft.filter(items=['Player', 'Year'])
#draft.rename(columns = {'Year' : 'Draft_Year'}, inplace=True)

#stats7 = stats6.merge(draft, how = 'left', on = 'Player')
#stats7.isnull().sum()[stats7.isnull().sum() > 0]
#stats7[stats7.Draft_Year.isnull()]

## III. Finalizing Player Information

### Incorporating All-Star Appearance

In [61]:
ast = pd.read_csv('../data/all_star_appearances.csv')
ast = ast.filter(items=['Player', '#']).rename(columns={'#' : 'n_allstar'})
ast.head()

,Player,n_allstar
0,Kareem Abdul-Jabbar*[a],19
1,LeBron James^,19
2,Kobe Bryant*,18
3,Tim Duncan*,15
4,Kevin Garnett*,15


In [62]:
ast['Player'] = ast['Player'].str.replace(r'[@#$%^&*}}[]]^[a]]', '')
ast.head()

,Player,n_allstar
0,Kareem Abdul-Jabbar*[a],19
1,LeBron James^,19
2,Kobe Bryant*,18
3,Tim Duncan*,15
4,Kevin Garnett*,15


In [63]:
ast['Player'] = ast['Player'].str.replace(r'*', '')
ast['Player'] = ast['Player'].str.replace(r'^', '')
ast['Player'] = ast['Player'].str.replace(r'[[a]]', '')
ast['Player'] = ast['Player'].str.replace(r'[', '')
ast['Player'] = ast['Player'].str.replace(r']', '')

In [64]:
ast

,Player,n_allstar
0,Kareem Abdul-Jabbar,19
1,LeBron James,19
2,Kobe Bryant,18
3,Tim Duncan,15
4,Kevin Garnett,15
...,...,...
445,Jayson Williams,1
446,Mo Williams,1
447,Kevin Willis,1
448,Metta World Peaced,1


In [65]:
stats6['Player'] = stats6['Player'].str.replace(r'*', '')
stats6['Player'] = stats6['Player'].str.replace(r'^', '')
stats6['Player'] = stats6['Player'].str.replace(r'[[a]]', '')
stats6['Player'] = stats6['Player'].str.replace(r'[', '')
stats6['Player'] = stats6['Player'].str.replace(r']', '')

In [66]:
stats6 = stats6.merge(ast, how='left', on='Player')
stats6.isnull().sum()[stats6.isnull().sum() > 0]

pg_3P%        156
tot_3P%       156
n_allstar    2537
dtype: int64

In [67]:
stats6['n_allstar'] = np.where(stats6.n_allstar.isnull(), 0, stats6['n_allstar'])

In [68]:
stats6.n_allstar.value_counts()

0.0     2537
1.0      516
2.0      287
3.0      165
6.0      129
5.0      107
4.0      101
10.0      97
7.0       67
8.0       63
15.0      54
9.0       52
11.0      44
14.0      41
13.0      29
12.0      27
19.0      20
18.0      17
Name: n_allstar, dtype: int64

In [69]:
stats6.isnull().sum()[stats6.isnull().sum() > 0]

pg_3P%     156
tot_3P%    156
dtype: int64

### Integrating Player Salaries

In [70]:
sal = pd.read_csv('../data/salaries.csv')
sal.head()

,Player,Salary,Salary_Adj,Year
0,Patrick Ewing,"$4,250,000","$9,694,547",1990
1,Hot Rod Williams,"$3,785,000","$8,633,850",1990
2,Hakeem Olajuwon,"$3,175,000","$7,242,397",1990
3,Charles Barkley,"$2,900,000","$6,615,103",1990
4,Chris Mullin,"$2,850,000","$6,501,049",1990


In [71]:
sal['Salary'] = sal['Salary'].str.replace('[$,]', '').astype(float)
sal['Salary_Adj'] = sal['Salary_Adj'].str.replace('[$,]', '').astype(float)

In [72]:
# Many names needed to updated in order to merge; merge was conducted and then external data manipulation in Excel and SQL helped to find a list of mismatching names
name_updates = {
    'PJ Brown': 'P.J. Brown',
    'BJ Armstrong': 'B.J. Armstrong',
    'Predrag Stojakovic': 'Peja Stojakovic',
    'Amare Stoudemire': "Amar'e Stoudemire",
    'TJ Ford': "T.J. Ford",
    'DJ Augustin': "D.J. Augustin",
    'OJ Mayo': "O.J. Mayo",
    'JR Smith': "J.R. Smith",
    'Hidayet Turkoglu': "Hedo Turkoglu",
    'Maurice Williams': "Mo Williams",
    'CJ Watson': "C.J. Watson",
    'JJ Hickson': "J.J. Hickson",
    'JJ Redick': "J.J. Redick",
    'PJ Tucker': "P.J. Tucker",
    'CJ Miles' : 'C.J. Miles',
    'Otto Porter': 'Otto Porter Jr.',
    'Tim Hardaway Jr': 'Tim Hardaway Jr.',
    'TJ McConnell': 'T.J. McConnell',
    'Tim Hardaway Jr': 'Tim Hardaway Jr.',
    'Dennis Schroeder': 'Dennis Schroder',
    'TJ Warren' : 'T.J. Warren',
    'Kelly Oubre' : 'Kelly Oubre Jr.',
    'Dennis Smith' : 'Dennis Smith Jr.',
    'Jaren Jackson Jr' : 'Jaren Jackson Jr.',
    'Larry Nance Jr' : 'Larry Nance Jr.',
    'Wendell Carter' : 'Wendell Carter Jr.',
    "Devonte Graham" : "Devonte' Graham",
    'Danuel House' : 'Danuel House Jr.', 
    'PJ Washington' : 'P.J. Washington', 
    'Michael Porter': 'Michael Porter Jr.',
    'Gary Trent Jr' : 'Gary Trent Jr.', 
    'Herb Jones' : 'Herbert Jones', 
    'Kevin Porter' : 'Kevin Porter Jr.',
    'Nicolas Claxton' : 'Nic Claxton', 
    'Kenyon Martin Jr' : 'Kenyon Martin Jr.', 
    'Trey Murphy' : 'Trey Murphy III',
    'Moe Harkless' : 'Maurice Harkless',
    'Louis Williams' : 'Lou Williams',
    'Radoslav Nesterovic' : 'Rasho Nesterovic',
    'Omer Asik' : 'Omer Asık',
    'BJ Mullens' : 'Byron Mullens',
    'Enes Kanter' : 'Enes Freedom',
    'Ishmael Smith': 'Ish Smith',
    'Jabari Smith' : 'Jabari Smith Jr.', 
    'Jabari Smith Sr': 'Jabari Smith',   
    'Marvin Bagley' : 'Marvin Bagley III'
}

In [73]:
# Source: https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string, ChatGPT
import unicodedata
def rmv_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

sal['Player'] = sal.Player.apply(rmv_accents)
sal['Player'] = sal['Player'].apply(lambda x: name_updates[x] if x in name_updates else x)

stats6['Player'] = stats6.Player.apply(rmv_accents)

In [74]:
stats7 = stats6.merge(sal, how = 'left', on = ['Player', 'Year'])

In [75]:
stats7.isnull().sum()[stats7.isnull().sum() > 0]

pg_3P%        156
tot_3P%       156
Salary         13
Salary_Adj     13
dtype: int64

In [76]:
# Imputation will be needed for these missing Salaries
stats7[stats7.Salary.isnull()]

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,GT1_Pos,Pos_5,PF,PG,SF,SG,Midseason_Trade,All_NBA_Team,Pts Won,Pts Max,Share,All_NBA_Winner,n_allstar,Salary,Salary_Adj
79,371,Reggie Williams,SF,26,DEN,73,46.0,26,5.3,11.7,0.449,0.8,2.2,0.363,4.5,9.6,0.468,0.482,2.3,2.7,0.843,1.8,2.4,4.2,1.8,1.5,0.6,1.5,3.5,13.6,1990,46.0,1896,384.0,855.0,0.449,57.0,157.0,0.363,327.0,698.0,0.468,0.482,166.0,197.0,0.843,133.0,173.0,306.0,133.0,113.0,41.0,112.0,253.0,991.0,14.8,0.526,0.184,0.230,6.3,9.6,7.9,9.4,2.6,1.2,10.6,20.2,2.2,1.0,3.3,0.082,0.0,0.1,0.1,1.0,0,SF,0,0,1,0,1,0,0.0,0.0,0.000,0.0,0.0,NaN,NaN
92,40,Anthony Bowie,SG,28,ORL,52,26.0,33,6.0,12.2,0.493,0.3,0.8,0.386,5.7,11.3,0.501,0.506,2.3,2.6,0.860,1.3,3.4,4.7,3.1,1.1,0.7,2.1,1.9,14.6,1991,26.0,1721,312.0,633.0,0.493,17.0,44.0,0.386,295.0,589.0,0.501,0.506,117.0,136.0,0.860,70.0,175.0,245.0,163.0,55.0,38.0,107.0,101.0,758.0,14.7,0.547,0.070,0.215,4.4,11.7,8.0,14.9,1.6,1.4,13.4,19.3,1.8,1.1,2.9,0.082,0.7,0.1,0.8,1.2,0,SG,0,0,0,1,0,0,0.0,0.0,0.000,0.0,0.0,NaN,NaN
317,386,Hot Rod Williams,PF,31,CLE,76,72.0,35,5.2,10.9,0.478,0.0,0.0,NaN,5.2,10.9,0.478,0.478,3.3,4.6,0.728,2.7,4.8,7.6,2.5,1.0,1.7,1.8,2.9,13.7,1993,72.0,2660,394.0,825.0,0.478,0.0,0.0,NaN,394.0,825.0,0.478,0.478,252.0,346.0,0.728,207.0,368.0,575.0,193.0,78.0,130.0,139.0,219.0,1040.0,16.2,0.532,0.000,0.419,9.0,16.1,12.6,11.3,1.5,3.2,12.5,18.8,3.8,3.6,7.4,0.134,0.6,0.9,1.5,2.4,0,PF,1,0,0,0,0,0,0.0,0.0,0.000,0.0,0.0,NaN,NaN
400,389,Hot Rod Williams,PF,32,CLE,74,73.0,35,4.9,10.9,0.452,0.0,0.1,0.200,4.9,10.9,0.453,0.452,2.6,3.9,0.685,2.3,4.5,6.9,2.6,1.1,1.4,2.0,2.9,12.6,1994,73.0,2641,366.0,810.0,0.452,1.0,5.0,0.200,365.0,805.0,0.453,0.452,196.0,286.0,0.685,173.0,334.0,507.0,192.0,83.0,101.0,149.0,211.0,929.0,14.6,0.496,0.006,0.353,7.9,16.3,12.0,13.2,1.8,3.1,13.7,19.7,0.9,4.4,5.3,0.096,-0.8,1.7,0.8,1.9,0,PF,1,0,0,0,0,0,0.0,0.0,0.000,0.0,0.0,NaN,NaN
494,411,Hot Rod Williams,C,33,PHO,62,58.0,26,2.9,6.4,0.453,0.0,0.0,0.000,2.9,6.4,0.455,0.453,1.5,2.1,0.731,2.1,3.9,6.0,1.0,0.7,1.5,1.0,2.7,7.3,1995,58.0,1652,180.0,397.0,0.453,0.0,1.0,0.000,180.0,396.0,0.455,0.453,95.0,130.0,0.731,129.0,243.0,372.0,62.0,46.0,90.0,62.0,170.0,455.0,12.4,0.501,0.003,0.327,9.2,16.8,13.0,5.5,1.4,4.1,12.0,13.9,1.0,1.7,2.8,0.080,-2.0,0.3,-1.6,0.2,0,C,0,0,0,0,0,0,0.0,0.0,0.000,0.0,0.0,NaN,NaN
588,423,Hot Rod Williams,C,34,PHO,68,66.0,31,3.0,6.1,0.490,0.0,0.0,0.000,3.0,6.1,0.493,0.490,2.0,2.9,0.672,2.6,5.6,8.3,1.5,1.0,1.3,1.0,2.6,8.0,1996,66.0,2137,204.0,416.0,0.490,0.0,2.0,0.000,204.0,414.0,0.493,0.490,133.0,198.0,0.672,178.0,384.0,562.0,100.0,67.0,88.0,66.0,176.0,541.0,13.8,0.538,0.005,0.476,9.8,20.8,15.4,6.7,1.6,3.1,11.6,12.0,2.8,2.5,5.3,0.119,-0.6,0.6,0.0,1.0,0,C,0,0,0,0,0,0,0.0,0.0,0.000,0.0,0.0,NaN,NaN
715,263,Sam Mitchell,SF,35,MIN,50,20.0,26,4.3,10.4,0.408,0.2,0.8,0.237,4.1,9.7,0.421,0.417,2.5,3.3,0.764,1.1,2.5,3.6,2.0,0.7,0.3,0.7,2.2,11.2,1998,20.0,1344,213.0,522.0,0.408,9.0,38.0,0.237,204.0,484.0,0.421,0.417,126.0,165.0,0.764,55.0,127.0,182.0,98.0,35.0,16.0,34.0,111.0,561.0,14.1,0.472,0.073,0.316,4.2,11.5,7.6,12.0,1.4,0.9,5.4,20.6,1.9,1.1,3.0,0.108,-0.4,-0.4,-0.9,0.4,0,SF,0,0,1,0,0,0,0.0,0.0,0.000,0.0,0.0,NaN,NaN
728,355,Dickey Simpkins,C,26,CHI,50,35.0,29,3.0,6.5,0.463,0.0,0.0,0.000,3.0,6.5,0.464,0.463,3.1,4.8,0.645,2.2,4.6,6.8,1.3,0.7,0.3,1.4,2.6,9.1,1998,35.0,1448,150.0,324.0,0.463,0.0,1.0,0.000,150.0,323.0,0.464,0.463,156.0,242.0,0.645,110.0,229.0,339.0,65.0,36.0,13.0,72.0,128.0,456.0,13.4,0.530,0.003,0.747,8.6,19.6,13.8,8.4,1.4,0.7,14.3,16.3,1.5,1.3,2.9,0

In [77]:
# Will fill in missing salaries for a player with their average salary
miss_sal = stats7[stats7['Salary'].isnull()]['Player'].unique()
fix_stats = stats7[stats7['Player'].isin(miss_sal)]
fix_stats.sort_values(by=['Player', 'Salary'], ascending=False, inplace=True)

avg_sals_for_miss = fix_stats.groupby('Player')['Salary', 'Salary_Adj'].mean().reset_index()

In [78]:
avg_sals_for_miss.columns = ['Player', 'Avg_Salary', 'Avg_Salary_Adj']
avg_sals_for_miss

,Player,Avg_Salary,Avg_Salary_Adj
0,Anthony Bowie,NaN,NaN
1,Clifford Robinson,4.460312e+06,7.924621e+06
2,Dickey Simpkins,NaN,NaN
3,Hot Rod Williams,3.785667e+06,8.294763e+06
4,Ira Newble,5.874350e+05,9.675560e+05
5,Kenyon Martin Jr.,NaN,NaN
6,Magic Johnson,2.400000e+06,5.474568e+06
7,Reggie Williams,1.058875e+06,2.181871e+06
8,Rod Strickland,2.899600e+06,5.364929e+06
9,Sam Mitchell,8.825000e+05,1.761010e+06


In [79]:
# For Anthony Bowie (1991) we will fill in the average NBA salary for 1991; source: https://www.thehoopsgeek.com/average-nba-salary/
avg_sals_for_miss['Avg_Salary'] = np.where((avg_sals_for_miss['Player'] == 'Anthony Bowie') & (avg_sals_for_miss['Avg_Salary'].isnull()), 1000000, avg_sals_for_miss['Avg_Salary'])
avg_sals_for_miss.loc[avg_sals_for_miss.Player == 'Anthony Bowie', 'Avg_Salary_Adj'] = 2240154

In [80]:
# For Dickey Simpkins (1998) we will fill in the average NBA salary for 1998
avg_sals_for_miss['Avg_Salary'] = np.where((avg_sals_for_miss['Player'] == 'Dickey Simpkins') & (avg_sals_for_miss['Avg_Salary'].isnull()), 2300000, avg_sals_for_miss['Avg_Salary'])
avg_sals_for_miss.loc[avg_sals_for_miss.Player == 'Dickey Simpkins', 'Avg_Salary_Adj'] = 4305219

# Kenyon Martin Jr's salary can be found externally and we will impute directly by Year - in future years, hopefully our reference site will include him in the tables we scrape
avg_sals_for_miss

,Player,Avg_Salary,Avg_Salary_Adj
0,Anthony Bowie,1.000000e+06,2.240154e+06
1,Clifford Robinson,4.460312e+06,7.924621e+06
2,Dickey Simpkins,2.300000e+06,4.305219e+06
3,Hot Rod Williams,3.785667e+06,8.294763e+06
4,Ira Newble,5.874350e+05,9.675560e+05
5,Kenyon Martin Jr.,NaN,NaN
6,Magic Johnson,2.400000e+06,5.474568e+06
7,Reggie Williams,1.058875e+06,2.181871e+06
8,Rod Strickland,2.899600e+06,5.364929e+06
9,Sam Mitchell,8.825000e+05,1.761010e+06


In [81]:
stats8 = stats7.merge(avg_sals_for_miss, how = 'left', on='Player')
stats8['Salary'].fillna(stats8['Avg_Salary'], inplace=True)
stats8['Salary_Adj'].fillna(stats8['Avg_Salary_Adj'], inplace=True)

# Kenyon Martin Jr
stats8.loc[(stats8.Player == 'Kenyon Martin Jr.') & (stats8.Year == 2020), 'Salary'] = 898310
stats8.loc[(stats8.Player == 'Kenyon Martin Jr.') & (stats8.Year == 2020), 'Salary_Adj'] = 1059003

stats8.loc[(stats8.Player == 'Kenyon Martin Jr.') & (stats8.Year == 2021), 'Salary'] = 1517981  
stats8.loc[(stats8.Player == 'Kenyon Martin Jr.') & (stats8.Year == 2021), 'Salary_Adj'] = 1709227 

stats8.loc[(stats8.Player == 'Kenyon Martin Jr.') & (stats8.Year == 2022), 'Salary'] = 1782621   
stats8.loc[(stats8.Player == 'Kenyon Martin Jr.') & (stats8.Year == 2022), 'Salary_Adj'] = 1858475 

stats8.drop(columns = ['Avg_Salary', 'Avg_Salary_Adj'], inplace=True)

In [82]:
stats8.isnull().sum()[stats8.isnull().sum() > 0]

pg_3P%     156
tot_3P%    156
dtype: int64

In [83]:
stats8[stats8.Salary.isnull()]

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,GT1_Pos,Pos_5,PF,PG,SF,SG,Midseason_Trade,All_NBA_Team,Pts Won,Pts Max,Share,All_NBA_Winner,n_allstar,Salary,Salary_Adj


## IV. Processing Team Information

### Conference and Win Percentages

In [84]:
team = pd.read_csv('../data/team_rank.csv')
team = team.filter(items=['Team', 'Conf', 'Div', 'W', 'L', 'W/L%', 'Year'])
team.head()

,Team,Conf,Div,W,L,W/L%,Year
0,Chicago Bulls,E,C,61,21,0.744,1990
1,Portland Trail Blazers,W,P,63,19,0.768,1990
2,Los Angeles Lakers,W,P,58,24,0.707,1990
3,Phoenix Suns,W,P,55,27,0.671,1990
4,Boston Celtics,E,A,56,26,0.683,1990


### Championship Winners

In [85]:
# Championship Team - Made this dictionary by heart :)
# Years represent season start not season end
nba_champions = {
    1989: 'Detroit Pistons',
    1990: 'Chicago Bulls',
    1991: 'Chicago Bulls',
    1992: 'Chicago Bulls',
    1993: 'Houston Rockets',
    1994: 'Houston Rockets',
    1995: 'Chicago Bulls',
    1996: 'Chicago Bulls',
    1997: 'Chicago Bulls',
    1998: 'San Antonio Spurs',
    1999: 'Los Angeles Lakers',
    2000: 'Los Angeles Lakers',
    2001: 'Los Angeles Lakers',
    2002: 'San Antonio Spurs',
    2003: 'Detroit Pistons',
    2004: 'San Antonio Spurs',
    2005: 'Miami Heat',
    2006: 'San Antonio Spurs',
    2007: 'Boston Celtics',
    2008: 'Los Angeles Lakers',
    2009: 'Los Angeles Lakers',
    2010: 'Dallas Mavericks',
    2011: 'Miami Heat',
    2012: 'Miami Heat',
    2013: 'San Antonio Spurs',
    2014: 'Golden State Warriors',
    2015: 'Cleveland Cavaliers',
    2016: 'Golden State Warriors',
    2017: 'Golden State Warriors',
    2018: 'Toronto Raptors',
    2019: 'Los Angeles Lakers',
    2020: 'Milwaukee Bucks',
    2021: 'Golden State Warriors',
    2022: 'Denver Nuggets'
}

team['Champs'] = team.Year.map(nba_champions)
team['Won_Championship'] = np.where(team.Champs == team.Team, 1, 0) # Flagging if player won championship that season

### Salary Cap

In [86]:
salcap = pd.read_csv('../data/salarycap.csv')
salcap.head()

,Year,Salary Cap,2022 Dollars
0,1984-85,"$3,600,000","$9,793,069"
1,1985-86,"$4,233,000","$11,298,280"
2,1986-87,"$4,945,000","$12,734,751"
3,1987-88,"$6,164,000","$15,248,702"
4,1988-89,"$7,232,000","$17,069,461"


In [87]:
salcap['Year'] = salcap.Year.str[:4].astype(int)
salcap['Salary_Cap'] = salcap['Salary Cap'].str.replace('[$,]', '').astype(float)
salcap['Salary_Cap_Adj'] = salcap['2022 Dollars'].str.replace('[$,]', '').astype(float)
salcap.loc[salcap.Salary_Cap_Adj.isnull(), 'Salary_Cap_Adj'] = salcap['Salary_Cap']
salcap.drop(columns=['Salary Cap', '2022 Dollars'], inplace=True)

In [88]:
salcap.dtypes

Year                int32
Salary_Cap        float64
Salary_Cap_Adj    float64
dtype: object

In [89]:
salcap.head()

,Year,Salary_Cap,Salary_Cap_Adj
0,1984,3600000.0,9793069.0
1,1985,4233000.0,11298280.0
2,1986,4945000.0,12734751.0
3,1987,6164000.0,15248702.0
4,1988,7232000.0,17069461.0


In [90]:
team = team.merge(salcap, how = 'left', on = 'Year')
team.head()

,Team,Conf,Div,W,L,W/L%,Year,Champs,Won_Championship,Salary_Cap,Salary_Cap_Adj
0,Chicago Bulls,E,C,61,21,0.744,1990,Chicago Bulls,1,11871000.0,25499592.0
1,Portland Trail Blazers,W,P,63,19,0.768,1990,Chicago Bulls,0,11871000.0,25499592.0
2,Los Angeles Lakers,W,P,58,24,0.707,1990,Chicago Bulls,0,11871000.0,25499592.0
3,Phoenix Suns,W,P,55,27,0.671,1990,Chicago Bulls,0,11871000.0,25499592.0
4,Boston Celtics,E,A,56,26,0.683,1990,Chicago Bulls,0,11871000.0,25499592.0


In [91]:
team.isnull().sum()

Team                0
Conf                0
Div                 0
W                   0
L                   0
W/L%                0
Year                0
Champs              0
Won_Championship    0
Salary_Cap          0
Salary_Cap_Adj      0
dtype: int64

### Team Payroll

In [92]:
payroll = pd.read_csv('../data/team_payroll.csv')
payroll['Team'].unique()

array(['Cleveland', 'New York', 'Detroit', 'LA Lakers', 'Atlanta',
       'Dallas', 'Philadelphia', 'Milwaukee', 'Phoenix', 'Brooklyn',
       'Boston', 'Portland', 'Golden State', 'San Antonio', 'Indiana',
       'Utah', 'Oklahoma City', 'Houston', 'Charlotte', 'Denver',
       'LA Clippers', 'Chicago', 'Washington', 'Sacramento', 'Miami',
       'Minnesota', 'Orlando', 'Memphis', 'Toronto', 'New Orleans'],
      dtype=object)

In [93]:
nba_payroll_map = {
    'Cleveland': 'Cleveland Cavaliers',
    'New York': 'New York Knicks',
    'Detroit': 'Detroit Pistons',
    'LA Lakers': 'Los Angeles Lakers',
    'Atlanta': 'Atlanta Hawks',
    'Dallas': 'Dallas Mavericks',
    'Philadelphia': 'Philadelphia 76ers',
    'Milwaukee': 'Milwaukee Bucks',
    'Phoenix': 'Phoenix Suns',
    'Brooklyn': 'Brooklyn Nets',
    'Boston': 'Boston Celtics',
    'Portland': 'Portland Trail Blazers',
    'Golden State': 'Golden State Warriors',
    'San Antonio': 'San Antonio Spurs',
    'Indiana': 'Indiana Pacers',
    'Utah': 'Utah Jazz',
    'Oklahoma City': 'Oklahoma City Thunder',
    'Houston': 'Houston Rockets',
    'Charlotte': 'Charlotte Hornets',
    'Denver': 'Denver Nuggets',
    'LA Clippers': 'Los Angeles Clippers',
    'Chicago': 'Chicago Bulls',
    'Washington': 'Washington Wizards',
    'Sacramento': 'Sacramento Kings',
    'Miami': 'Miami Heat',
    'Minnesota': 'Minnesota Timberwolves',
    'Orlando': 'Orlando Magic',
    'Memphis': 'Memphis Grizzlies',
    'Toronto': 'Toronto Raptors',
    'New Orleans': 'New Orleans Pelicans'
}

In [94]:
payroll['Team'] = payroll.Team.map(nba_payroll_map)

# Update for Charlotte's temporary rename
overwrite = ((payroll['Team'] == 'Charlotte Hornets') & (payroll['Year'].between(2004, 2013)))
payroll.loc[overwrite, 'Team'] = 'Charlotte Bobcats'

In [95]:
team = team.merge(payroll, how = 'left', on = ['Team', 'Year'])
team.head()

,Team,Conf,Div,W,L,W/L%,Year,Champs,Won_Championship,Salary_Cap,Salary_Cap_Adj,Payroll,Payroll_Adj
0,Chicago Bulls,E,C,61,21,0.744,1990,Chicago Bulls,1,11871000.0,25499592.0,"$10,040,000","$22,901,939"
1,Portland Trail Blazers,W,P,63,19,0.768,1990,Chicago Bulls,0,11871000.0,25499592.0,"$11,215,000","$25,582,195"
2,Los Angeles Lakers,W,P,58,24,0.707,1990,Chicago Bulls,0,11871000.0,25499592.0,"$12,120,000","$27,646,565"
3,Phoenix Suns,W,P,55,27,0.671,1990,Chicago Bulls,0,11871000.0,25499592.0,"$11,463,000","$26,147,902"
4,Boston Celtics,E,A,56,26,0.683,1990,Chicago Bulls,0,11871000.0,25499592.0,"$11,256,000","$25,675,719"


In [96]:
# Some payrolls are missing - fill them in with this list of salaries sourced from text files made by Patricia Bender: https://www.eskimo.com/~pbender/misc/salaries&yr..txt where &yr. = the last two digits of the season end year ('91 to '23)
missing_payroll = [
    
    (1990, 'New Jersey Nets', 11860000, 2.33),
    (1990, 'Seattle SuperSonics', 10890000, 2.33),
    (1990, 'Washington Bullets', 9640000, 2.33),
    
    (1991, 'Washington Bullets', 12633000, 2.24),
    (1991, 'New Jersey Nets', 12598000, 2.24),
    (1991, 'Seattle SuperSonics', 13308000, 2.24),
    (1992, 'Washington Bullets', 17249000, 2.17),
    (1992, 'Seattle SuperSonics', 14033000, 2.17),
    (1992, 'New Jersey Nets', 16314000, 2.17),
    (1993, 'New Jersey Nets', 20100000, 2.11),
    (1993, 'Washington Bullets', 16100000, 2.11),
    (1993, 'Seattle SuperSonics', 16000000, 2.11),
    (1994, 'Seattle SuperSonics', 24073500, 2.06),
    (1994, 'Washington Bullets', 17049750, 2.06),
    (1994, 'New Jersey Nets', 23121800, 2.06),
    (1994, 'Vancouver Grizzlies', 18413000, 2.06),
    (1995, 'Vancouver Grizzlies', 18413000, 2),
    (1995, 'Washington Bullets', 22224000, 2),
    (1995, 'Seattle SuperSonics', 25852000, 2),
    (1995, 'New Jersey Nets', 22580000, 2),

    (1996, 'Vancouver Grizzlies', 18640000, 1.94),
    (1996, 'New Jersey Nets', 25430000, 1.94),
    (1996, 'Seattle SuperSonics', 30300000, 1.94),
    (1996, 'Washington Bullets', 34560000, 1.94),
    (1997, 'New Jersey Nets', 28504567, 1.9),
    (1997, 'Seattle SuperSonics', 37592175, 1.9),
    (1997, 'Vancouver Grizzlies', 25637110, 1.9),
    (1998, 'New Jersey Nets', 4196000, 1.87),
    (1998, 'Vancouver Grizzlies', 33340000, 1.87),
    (1998, 'Seattle SuperSonics', 31920000, 1.87),
    (1999, 'New Jersey Nets', 52735461, 1.83),
    (1999, 'Seattle SuperSonics', 38258910, 1.83),
    (1999, 'Vancouver Grizzlies', 37749568, 1.83),
    (2000, 'New Jersey Nets', 68977584, 1.77),
    (2000, 'Seattle SuperSonics', 50635656, 1.77),
    (2000, 'Vancouver Grizzlies', 48198340, 1.77),
    (2001, 'New Jersey Nets', 75172371, 1.72),
    (2001, 'Charlotte Hornets', 49999445, 1.72),
    (2001, 'Seattle SuperSonics', 45431733, 1.72),
    (2002, 'New Jersey Nets', 60350424, 1.7),
    (2002, 'New Orleans Hornets', 44458880, 1.7),
    (2002, 'Seattle SuperSonics', 53167480, 1.7),

    (2003, 'New Jersey Nets', 63273120, 1.66),
    (2003, 'New Orleans Hornets', 49037564, 1.66),
    (2003, 'Seattle SuperSonics', 52347480, 1.66),
    (2004, 'New Orleans Hornets', 56572880, 1.62),
    (2004, 'New Jersey Nets', 54729028, 1.62),
    (2004, 'Seattle SuperSonics', 53821300, 1.62),
    (2005, 'New Jersey Nets', 66030223, 1.56),
    (2005, 'New Orleans/Oklahoma City Hornets', 41277617, 1.56),
    (2005, 'Seattle SuperSonics', 48900280, 1.56),
    (2006, 'New Jersey Nets', 64372808, 1.51),
    (2006, 'New Orleans/Oklahoma City Hornets', 53711107, 1.51),
    (2006, 'Seattle SuperSonics', 56931964, 1.51),
    (2007, 'New Jersey Nets', 61366182, 1.47),
    (2007, 'New Orleans Hornets', 62346671, 1.47),
    (2007, 'Seattle SuperSonics', 60861388, 1.47),
    (2008, 'New Jersey Nets', 62666523, 1.42),
    (2008, 'New Orleans Hornets', 66842294, 1.42),
    (2009, 'New Jersey Nets', 59366715, 1.42),
    (2009, 'New Orleans Hornets', 69721295, 1.42),
    (2010, 'New Jersey Nets', 58494181, 1.4),
    (2010, 'New Orleans Hornets', 67985781, 1.4),
    (2011, 'New Orleans Hornets', 65418702, 1.36),
    (2011, 'New Jersey Nets', 60857455, 1.36),
    (2012, 'New Orleans Hornets', 63517041, 1.33)
]

missing_payroll = pd.DataFrame(missing_payroll, columns=['Year', 'Team', 'Missing_Payroll', 'Rough_Conversion']) # Dollar conversion source: https://www.in2013dollars.com/us/inflation
missing_payroll['Missing_Payroll_Adj'] = missing_payroll['Missing_Payroll']*missing_payroll['Rough_Conversion']
missing_payroll.drop(columns=['Rough_Conversion'], inplace=True)

In [97]:
missing_payroll.dtypes

Year                     int64
Team                    object
Missing_Payroll          int64
Missing_Payroll_Adj    float64
dtype: object

In [98]:
# Merge in the payroll figures to impute into the missings
team = team.merge(missing_payroll, how='left', on = ['Team', 'Year'])

# Use fillna to map, discard unneeded variables
team['Payroll'] = team['Payroll'].fillna(team['Missing_Payroll'])
team['Payroll_Adj'] = team['Payroll_Adj'].fillna(team['Missing_Payroll_Adj'])
team.drop(columns=['Missing_Payroll', 'Missing_Payroll_Adj'], inplace=True)

In [99]:
team.isnull().sum()

Team                0
Conf                0
Div                 0
W                   0
L                   0
W/L%                0
Year                0
Champs              0
Won_Championship    0
Salary_Cap          0
Salary_Cap_Adj      0
Payroll             0
Payroll_Adj         0
dtype: int64

## V. Merge Player and Team Data

In [100]:
nba_team_map = {
    'HOU': 'Houston Rockets',
    'MIA': 'Miami Heat',
    'POR': 'Portland Trail Blazers',
    'PHI': 'Philadelphia 76ers',
    'PHO': 'Phoenix Suns',
    'DEN': 'Denver Nuggets',
    'SAC': 'Sacramento Kings',
    'GSW': 'Golden State Warriors',
    'IND': 'Indiana Pacers',
    'ATL': 'Atlanta Hawks',
    'ORL': 'Orlando Magic',
    'LAC': 'Los Angeles Clippers',
    'BOS': 'Boston Celtics',
    'UTA': 'Utah Jazz',
    'MIL': 'Milwaukee Bucks',
    'DET': 'Detroit Pistons',
    'CLE': 'Cleveland Cavaliers',
    'NYK': 'New York Knicks',
    'LAL': 'Los Angeles Lakers',
    'DAL': 'Dallas Mavericks',
    'MIN': 'Minnesota Timberwolves',
    'CHI': 'Chicago Bulls',
    'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'WAS': 'Washington Wizards',
    'MEM': 'Memphis Grizzlies',
    'NJN': 'New Jersey Nets',
    'SEA': 'Seattle SuperSonics',
    'OKC': 'Oklahoma City Thunder',
    'NOP': 'New Orleans Pelicans',
    'CHO': 'Charlotte Hornets', # (2014-2018)
    'NOH': 'New Orleans Hornets', # (2002-2005)
    'BRK': 'Brooklyn Nets',
    'WSB': 'Washington Bullets',
    'VAN': 'Vancouver Grizzlies',
    'NOK': 'New Orleans/Oklahoma City Hornets',
    'CHH': 'Charlotte Hornets', # (1988-2002)
    'CHA': 'Charlotte Hornets', # (2014-present)
    'CHO': 'Charlotte Hornets', # (2014-2018)
    # 'CHA': 'Charlotte Bobcats' # (2004-2013)
}

In [101]:
stats8['Team'] = stats8.Tm.map(nba_team_map)

In [102]:
# Overwrite Charlotte Hornets to Charlotte Bobcats from 2004-2013
overwrite = (stats8['Team'] == 'Charlotte Hornets') & (stats8['Year'].between(2004, 2013))
stats8.loc[overwrite, 'Team'] = 'Charlotte Bobcats'
stats8[(stats8.Tm == 'CHA') & (stats8.Year.between(2004, 2013))].head()

,Rk,Player,Pos,Age,Tm,G,pg_GS,pg_MP,pg_FG,pg_FGA,pg_FG%,pg_3P,pg_3PA,pg_3P%,pg_2P,pg_2PA,pg_2P%,pg_eFG%,pg_FT,pg_FTA,pg_FT%,pg_ORB,pg_DRB,pg_TRB,pg_AST,pg_STL,pg_BLK,pg_TOV,pg_PF,pg_PTS,Year,tot_GS,tot_MP,tot_FG,tot_FGA,tot_FG%,tot_3P,tot_3PA,tot_3P%,tot_2P,tot_2PA,tot_2P%,tot_eFG%,tot_FT,tot_FTA,tot_FT%,tot_ORB,tot_DRB,tot_TRB,tot_AST,tot_STL,tot_BLK,tot_TOV,tot_PF,tot_PTS,adv_PER,adv_TS%,adv_3PAr,adv_FTr,adv_ORB%,adv_DRB%,adv_TRB%,adv_AST%,adv_STL%,adv_BLK%,adv_TOV%,adv_USG%,adv_OWS,adv_DWS,adv_WS,adv_WS/48,adv_OBPM,adv_DBPM,adv_BPM,adv_VORP,GT1_Pos,Pos_5,PF,PG,SF,SG,Midseason_Trade,All_NBA_Team,Pts Won,Pts Max,Share,All_NBA_Winner,n_allstar,Salary,Salary_Adj,Team
1173,53,Primoz Brezec,C,25,CHA,72,72.0,31,5.4,10.5,0.512,0.0,0.0,NaN,5.4,10.5,0.512,0.512,2.3,3.1,0.745,3.1,4.2,7.4,1.2,0.5,0.8,1.5,2.5,13.0,2004,72.0,2276,387.0,756.0,0.512,0.0,0.0,NaN,387.0,756.0,0.512,0.512,164.0,220.0,0.745,226.0,305.0,531.0,86.0,33.0,55.0,106.0,183.0,938.0,16.1,0.550,0.000,0.291,10.8,16.2,13.3,6.6,0.8,1.8,11.1,18.7,3.5,1.4,4.9,0.104,0.5,-1.5,-1.0,0.6,0,C,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1554326.0,2427853.0,Charlotte Bobcats
1212,244,Brevin Knight,PG,29,CHA,66,61.0,29,3.8,8.9,0.422,0.0,0.3,0.150,3.7,8.6,0.432,0.425,2.5,3.0,0.852,0.3,2.3,2.6,9.0,2.0,0.1,2.2,2.9,10.1,2004,61.0,1944,248.0,587.0,0.422,3.0,20.0,0.150,245.0,567.0,0.432,0.425,167.0,196.0,0.852,21.0,149.0,170.0,591.0,131.0,5.0,147.0,192.0,666.0,18.0,0.495,0.034,0.334,1.2,9.3,5.0,49.3,3.5,0.2,17.9,18.7,2.6,1.9,4.5,0.110,0.7,1.1,1.8,1.8,0,PG,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,932546.0,1456634.0,Charlotte Bobcats
1226,312,Emeka Okafor,PF,22,CHA,73,73.0,35,6.1,13.7,0.447,0.0,0.0,0.000,6.1,13.7,0.447,0.447,2.9,4.7,0.609,3.8,7.1,10.9,0.9,0.8,1.7,1.7,2.9,15.1,2004,73.0,2600,448.0,1003.0,0.447,0.0,1.0,0.000,448.0,1002.0,0.447,0.447,209.0,343.0,0.609,275.0,520.0,795.0,64.0,62.0,125.0,125.0,214.0,1105.0,16.3,0.479,0.001,0.342,11.5,24.1,17.5,4.3,1.2,3.6,9.8,21.8,0.9,3.1,4.0,0.074,-0.7,-1.3,-2.0,0.0,0,PF,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3739680.0,5841372.0,Charlotte Bobcats
1252,433,Gerald Wallace,SF,22,CHA,70,68.0,30,4.1,9.1,0.449,0.2,0.9,0.274,3.8,8.2,0.468,0.462,2.7,4.1,0.661,1.7,3.8,5.5,2.0,1.7,1.3,2.3,2.6,11.1,2004,68.0,2147,286.0,637.0,0.449,17.0,62.0,0.274,269.0,575.0,0.468,0.462,191.0,289.0,0.661,118.0,268.0,386.0,137.0,117.0,91.0,159.0,182.0,780.0,14.1,0.510,0.097,0.454,6.0,15.1,10.3,10.4,2.8,3.2,17.2,19.1,-0.2,2.7,2.4,0.055,-1.2,1.4,0.2,1.2,0,SF,0,0,1,0,0,0,0.0,0.0,0.0,0.0,1.0,1652453.0,2581128.0,Charlotte Bobcats
1268,54,Primoz Brezec,C,26,CHA,79,79.0,27,5.2,10.0,0.517,0.0,0.0,0.000,5.2,10.0,0.518,0.517,2.1,2.8,0.732,2.3,3.3,5.6,0.6,0.2,0.4,1.1,2.9,12.4,2005,79.0,2165,409.0,791.0,0.517,0.0,2.0,0.000,409.0,789.0,0.518,0.517,164.0,224.0,0.732,181.0,259.0,440.0,45.0,19.0,32.0,85.0,227.0,982.0,15.3,0.552,0.003,0.283,9.1,14.8,11.8,3.7,0.5,1.1,8.7,20.0,3.4,1.2,4.6,0.102,-0.4,-2.0,-2.4,-0.2,0,C,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2750000.0,4189487.0,Charlotte Bobcats


In [103]:
# Merge Team into Stats table
stats_main = stats8.merge(team, how = 'left', on = ['Team', 'Year'])
stats_main.drop(columns=['Rk'], inplace=True)

In [104]:
stats_main.columns = stats_main.columns.str.strip().str.lower().str.replace(" ", "_")

In [105]:
stats_main.isnull().sum()[stats_main.isnull().sum() > 0]

pg_3p%     156
tot_3p%    156
dtype: int64

In [106]:
stats_main.shape

(4353, 101)

In [107]:
stats_main.to_csv('../data/clean/stats_main.csv', index=False)